# KẾT HỢP CÁC FILE CHI PHÍ BCQT19

In [37]:
import os
import numpy as np
import pandas as pd
import openpyxl

# Specify the folder containing your Excel files
folder_path = r'D:\Bao Cao Chi Phí\BCQT_19'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Filter only Excel files (xlsx, xls, xlsm, etc.) that are not temporary files
excel_files = [file for file in file_list if (file.endswith('.xlsx') or file.endswith('.xls')) and not file.startswith('~$')]

# Create a dictionary to store DataFrames
dataframes = {}

# Loop through each Excel file and read its contents into a DataFrame
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, skiprows=2).fillna(0)  # Read the Excel file into a DataFrame, skipping the first 2 rows
    
    # Extract the month from the filename
    month = file.split('.')[0] if file.endswith('.xlsx') else 'Unknown'
    
    # Rename columns excluding specified columns
    columns_to_rename = [col for col in df.columns if col not in ["MA_DV", "TEN_TEN_VI", "PROFIT_CENTRE", "PROFIT_CENTRE_DESC"]]
    new_column_names = [f"{col}_{month}" for col in columns_to_rename]
    column_mapping = dict(zip(columns_to_rename, new_column_names))
    df = df.rename(columns=column_mapping)
    
    # Create the "code" column based on specified logic
    def create_code(row):
        if row['MA_DV'] in ["'001", "'179", "'379", "'323", "'324", "'166", "'188", "'666", "'688", "'668"]:
            code_prefix = 'HO'
        else:
            code_prefix = 'ĐVKD'

        code = f"{code_prefix}_{row['MA_DV'][-3:]}_{row['PROFIT_CENTRE'][-3:]}"
        return code

    df['code'] = df.apply(create_code, axis=1)

    # Drop the specified columns    df = df.drop(columns=['MA_DV', 'TEN_TEN_VI', 'PROFIT_CENTRE', 'PROFIT_CENTRE_DESC'])

    df=pd.melt(df, id_vars=['code','MA_DV', 'TEN_TEN_VI', 'PROFIT_CENTRE', 'PROFIT_CENTRE_DESC'], var_name="var", value_name='val')
    
    # Store the DataFrame in the dictionary
    dataframes[file] = df

# Concatenate the DataFrames from the dictionary into a single DataFrame
combined_dataframe = pd.concat(dataframes.values(), ignore_index=True)


In [38]:
pivot_df=combined_dataframe.pivot(index=['code','MA_DV','TEN_TEN_VI','PROFIT_CENTRE','PROFIT_CENTRE_DESC'], columns='var', values='val').reset_index()

# sắp xếp các cột theo thứ tự bên dưới
patterns = [
    "LUONG VA CAC PHU CAP",
    "T.PHUC, TRO CAP THOI VIEC",
    "CHI KHAC CHO CBNV",
    "VAT LIEU, GIAY TO IN",
    "CONG TAC PHI",
    "XANG DAU",
    "DAO TAO, HLNV, N.CUU",
    "MUA TAI LIEU, SACH BAO",
    "BUU PHI, DIEN THOAI",
    "XB TAI LIEU, QC, TT, KMAI",
    "HOAT DONG DOAN THE",
    "DIEN, NUOC, VE SINH",
    "Y TE CO QUAN",
    "HOI NGHI",
    "LE TAN KHANH TIET",
    "KIEM TOAN, THANH TRA",
    "BAO VE TAI SAN THONG THUONG",
    "BAO VE TS. XIET NO, G.NO",
    "PCCC",
    "T.VAN PHAP LUAT",
    "CHI PHI THU HOI NO",
    "CHI PHI VP, HDQT, BKS, DH CO DONG",
    "CHI KHAC",
    "PHI, LE PHI",
    "THUE GTGT KHONG DUOC KT",
    "KHAU HAO TSCD",
    "MUA SAM CCLD",
    "S.CHUA NHO, SC LON TSCD",
    "BAO HIEM TAI SAN",
    "THUE TAI SAN"
]
name_columns=['code','MA_DV','TEN_TEN_VI','PROFIT_CENTRE','PROFIT_CENTRE_DESC']
ordered_columns = [col for pattern in patterns for col in pivot_df.columns if pattern in col]
columns=name_columns+ordered_columns

In [39]:
TongHop_df=pivot_df[columns].copy()
TongHop_df.head()

var,code,MA_DV,TEN_TEN_VI,PROFIT_CENTRE,PROFIT_CENTRE_DESC,LUONG VA CAC PHU CAP_1,LUONG VA CAC PHU CAP_2,LUONG VA CAC PHU CAP_3,LUONG VA CAC PHU CAP_4,LUONG VA CAC PHU CAP_5,...,BAO HIEM TAI SAN_5,BAO HIEM TAI SAN_6,BAO HIEM TAI SAN_7,THUE TAI SAN_1,THUE TAI SAN_2,THUE TAI SAN_3,THUE TAI SAN_4,THUE TAI SAN_5,THUE TAI SAN_6,THUE TAI SAN_7
0,HO_001_011,'001,TRU SO CHINH,'011,HOI DONG QUAN TRI,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,101045089.0,320000000.0,350880000.0,556200000.0,339000000.0,339000000.0,320000000.0,537200000.0
1,HO_001_012,'001,TRU SO CHINH,'012,BAN KIEM SOAT,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,55000000.0,55000000.0,55000000.0,55000000.0,59475000.0,55000000.0,54000000.0
2,HO_001_021,'001,TRU SO CHINH,'021,PHONG KIEM TOAN NOI BO,1.010577e+09,2.034918e+09,2.041051e+09,2.039452e+09,2.026973e+09,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,HO_001_030,'001,TRU SO CHINH,'030,BAN TONG GIAM DOC,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.0,0.0,0.0,92620000.0,92620000.0,92620000.0,92620000.0,95988000.0,92620000.0,90936000.0
4,HO_001_031,'001,TRU SO CHINH,'031,BAN TONG GIAM DOC,1.151546e+09,2.618480e+09,3.203708e+09,2.747627e+09,2.632137e+09,...,0.0,0.0,0.0,84700000.0,244200000.0,249700000.0,293700000.0,243181000.0,354200000.0,187660000.0


In [40]:
ChiPhi_HO_df=TongHop_df[TongHop_df['code'].str.startswith('HO')]
ChiPhi_DVKD_df=TongHop_df[~TongHop_df['code'].str.startswith('HO')]

# 1. KHOẢN MỤC - THÁNG

In [41]:


# Specify the folder containing your Excel files
folder_path = r'D:\Bao Cao Chi Phí\BCQT_19'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)
# find the number of file in the folder
file_num=len(file_list)

# Filter only Excel files (xlsx, xls, xlsm, etc.) that are not temporary files
excel_files = [file for file in file_list if (file.endswith('.xlsx') or file.endswith('.xls')) and not file.startswith('~$')]

# Create a dictionary to store DataFrames
dataframes = {}

# Loop through each Excel file and read its contents into a DataFrame
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_excel(file_path, skiprows=2)  # Read the Excel file into a DataFrame, skipping the first 2 rows
    
    # Extract the month from the filename
    month = file.split('.')[0] if file.endswith('.xlsx') else 'Unknown'
    
    # Rename columns excluding specified columns
    columns_to_rename = [col for col in df.columns if col not in ["MA_DV", "TEN_TEN_VI", "PROFIT_CENTRE", "PROFIT_CENTRE_DESC"]]
    new_column_names = [f"{col}_{month}" for col in columns_to_rename]
    column_mapping = dict(zip(columns_to_rename, new_column_names))
    df = df.rename(columns=column_mapping)
    
    # Create the "code" column based on specified logic
    def create_code(row):
        if row['MA_DV'] in ["'001", "'179", "'379", "'323", "'324", "'166", "'188", "'666", "'688", "'668"]:
            code_prefix = 'HO'
        else:
            code_prefix = 'ĐVKD'

        code = f"{code_prefix}_{row['MA_DV'][-3:]}_{row['PROFIT_CENTRE'][-3:]}"
        return code

    df['code'] = df.apply(create_code, axis=1)

    # Drop the specified columns    
    df = df.drop(columns=['MA_DV', 'TEN_TEN_VI', 'PROFIT_CENTRE', 'PROFIT_CENTRE_DESC'])

    df=pd.melt(df, id_vars='code', var_name="var", value_name='val')
    
    # Store the DataFrame in the dictionary
    dataframes[file] = df

# Concatenate the DataFrames from the dictionary into a single DataFrame
combined_dataframe = pd.concat(dataframes.values(), ignore_index=True)
bcqt19_df=combined_dataframe.copy()

# TÁCH DỮ LIỆU THÀNH CÁC THÁNG VÀ CATEGORY

In [42]:
df=bcqt19_df.copy()
# Find the position of "_" in each value of the "column_name" column
df['underscore_position'] = df['var'].str.find('_')
# Create a new column "substring_column" by extracting substrings based on "underscore_position"
df['category'] = df.apply(lambda row: row['var'][:row['underscore_position']] , axis=1)
df['Month'] = df.apply(lambda row: row['var'][row['underscore_position']+1:] , axis=1)
df=df.groupby(['category','Month']).val.sum().reset_index()
df['val']=df['val']/1000000000
data=df.copy()


# CỘNG THÊM CHI PHÍ CỘNG TAY Ở NGOÀI CHO CHI KHÁC

In [43]:
import pandas as pd

folder_path = r'D:\\Bao Cao LNK\\LNK\\'
sheet_names = [
    'LN T1 (ĐC DV các khối)',
    'LN T2 (ĐC DV các khối)',
    'LN T3 (ĐC DV các khối)',
    'LN T4 (ĐC DV các khối)',
    'LN T5 (ĐC DV các khối)',
    'LN T6 (ĐC DV các khối)',
    'LN T7 (ĐC DV các khối)'
]

months = []
vals = []

for i, sheet_name in enumerate(sheet_names, start=1):
    file_path = f'{folder_path}LNK {i}.xlsx'

    try:
        df = pd.read_excel(file_path, sheet_name=sheet_name)
        # lấy ô tổng THUẾ MB, GTGT, KHNH, TNDN + CHI PHÍ DỰ PHÒNG GIẢM GIÁ VÀNG, HÀNG TỒN KHO…
        cell_value = (df.iloc[124, 12] + df.iloc[125, 12]) / 1e9  # Divide by 1 billion to convert to billions
        months.append(i)
        vals.append(cell_value)
    except Exception as e:
        print(f'Error reading file LNK {i}.xlsx with sheet name "{sheet_name}": {str(e)}')

print(f'Các tháng trong file: {months}')
print(f'Các giá trị cộng thêm: {vals}')

ChiPhiKhac_df=pd.DataFrame({
    'Month': months,
    'val': vals,
    'category':'CHI KHAC'
})
ChiPhiKhac_df.Month=ChiPhiKhac_df.Month.astype(str)

# nối bảng chi phí khác vào bảng data
merged_df=pd.merge(data,ChiPhiKhac_df,on=['Month','category'],how='outer').fillna(0)
# cộng chi phí khác vs chi phí ngoài cộng tay
merged_df['val']=merged_df['val_x']+merged_df['val_y']
data=merged_df.drop(columns=['val_x','val_y'])

Các tháng trong file: [1, 2, 3, 4, 5, 6, 7]
Các giá trị cộng thêm: [0.348, 0.011022156, 0.00326388, 0.002634576, 0.001564505, 0.107418023, 0.003722448]


# TẠO BẢO CÁO CHI PHÍ TỪNG THÁNG THEO KHOẢN MỤC NHỎ

In [44]:
file_path=r'D:\Bao Cao Chi Phí\info-ChiPhi.xlsx'
ChiPhiTaiKhoan_df=pd.read_excel(file_path,sheet_name='Tai Khoan 1')
df1=ChiPhiTaiKhoan_df.copy()
# CÁC LOẠI CHI PHÍ (đ/c)
df1=df1[['category','Loại chi phí (đ/c)']]
df1=df1[df1.category.notnull()]
q1=data.merge(df1).groupby(['Loại chi phí (đ/c)','Month']).val.sum().reset_index().rename(columns={'Loại chi phí (đ/c)':'Khoản mục'})


# CÁC KHOẢN MỤC LỚN: Chi phí cho CBNV, Chi phí marketing, Chi phí hoạt động quản lý...

In [45]:
# CÁC LOẠI KHOẢN MỤC CHI PHÍ
df2=ChiPhiTaiKhoan_df.copy()
df2=df2[['category','Khoản mục']]
df2=df2[df2['Khoản mục'].notnull()]
q2=df2.merge(data).groupby(['Khoản mục','Month']).val.sum().reset_index()

# TÁCH LƯƠNG KINH DOANH BẢO HIỂM KHỎI CHI CHO CBNV


In [46]:
file_path=r'D:\Bao Cao Chi Phí\N2023 - LƯƠNG KD - PC.xlsx'
df=pd.read_excel(file_path,sheet_name='N2023 Lương TH',skiprows=1)
# KIỂM TRA NGÀY CHI LƯƠNG CÁC MÓN KD BẢO HIỂM
df.iloc[0:2,4:20]
# => GỘP 2 MÓN THÁNG 4 LẠI VỚI NHAU và THÁNG 3 = 0

,Tên ĐVKD,T12.2022,T01.2023,T01.2023.1,T02.2023,T03.2023,T04.2023,T05.2023,T06.2023,T07.2023,T08.2023,T09.2023,T10.2023,T11.2023,Lũy kế
0,Ngày chi lương,16/01/2023,25/2/2023,03/04/2023,2023-04-25 00:00:00,2023-05-25 00:00:00,2023-06-25 00:00:00,2023-07-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,Năm 2022
1,Tổng,58210294350,2336878813,27782361299,43599058772,30013216525,8560083118,7128860083,0.0,0.0,0.0,0.0,0.0,0.0,177630752960


In [47]:
file_path=r'D:\Bao Cao Chi Phí\N2023 - LƯƠNG KD - PC.xlsx'
df=pd.read_excel(file_path,sheet_name='N2023 Lương TH',skiprows=1)
# KIỂM TRA NGÀY CHI LƯƠNG CÁC MÓN KD BẢO HIỂM
df.iloc[0:2,4:20]
# => GỘP 2 MÓN THÁNG 4 LẠI VỚI NHAU và THÁNG 3 = 0

# TẠO DANH SÁCH CÁC THÁNG
month_list=[]
for i in range(1,13):
    month_list.append(i)

# TẠO DANH SÁCH CHI PHÍ KD BẢO HIỂM
val_list=[]
for i in df.iloc[1,4:20][1:].values:
    val_list.append(i)
del val_list[13]
del val_list[12]
# Số liệu tháng 4 bằng element 
sum_value = val_list[2]+val_list[3]
# Remove elements at positions 1 and 2
del val_list[2]
del val_list[2]

# Số liệu tháng 3 = 0
new_value=0
val_list.insert(2, new_value)
# Số liệu tháng 4 mới
val_list.insert(3, sum_value)

# Creating a DataFrame
two_list = {'Month': month_list, 'val': val_list}
KDBH_df = pd.DataFrame(two_list)
df=KDBH_df.copy()
df['val']=df['val']/1e9
df['Month']=df['Month'].astype(str)
# thêm cột khoản mục Chi phí cho CBNV để nối bảng
df['Khoản mục']='Chi phí cho CBNV'

merged_df=pd.merge(q2,df,on=['Month','Khoản mục'],how='outer').fillna(0)
merged_df['val']=merged_df['val_x']-merged_df['val_y']
q2=merged_df.drop(columns=['val_x','val_y'])
q2.Month=q2.Month.astype(int)

q2=q2[q2.Month<=file_num]
q2['Month']=q2['Month'].astype(str)

q5=KDBH_df
q5['val']=q5['val']/1e9
q5=q5[q5.Month<=file_num]
# chi phí lương kinh doanh bảo hiểm
q5['Khoản mục']='Chi phí lương kinh doanh bảo hiểm'
q5.Month=q5.Month.astype(str)

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\1117925140.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q5['Khoản mục']='Chi phí lương kinh doanh bảo hiểm'
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\1117925140.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  q5.Month=q5.Month.astype(str)


# TÍNH TỔNG CHI PHÍ QUẢN LÝ

In [48]:
# TỔNG CHI PHÍ QUẢN LÝ = chí phí HDQT vs BKS, chi phí cho CBNV, chi phí hoạt động quản lý, chi phí marketing và chi phí về tài sản
q3=q2.groupby('Month').val.sum().reset_index()
q3['Khoản mục']='Tổng chi phí quản lý'
q3['Month']=q3['Month'].astype(str)

# LẤY SỐ Chi phí bảo hiểm, bảo toàn tiền gửi

In [49]:
# Specify the folder containing your Excel files
folder_path = r'D:\Bao Cao Chi Phí\BCQT_10'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# find the number of file in the folder
file_num=len(file_list)

# Filter only Excel files (xlsx, xls, xlsm, etc.) that are not temporary files
excel_files = [file for file in file_list if (file.endswith('.xlsx') or file.endswith('.xls')) and not file.startswith('~$')]

# Create a dictionary to store DataFrames
dataframes = {}
months=[]
vals=[]
# Loop through each Excel file and read its contents into a DataFrame
for file in excel_files:
    file_path = os.path.join(folder_path, file)
    # Extract the month from the filename
    month = file.split('.')[0] if file.endswith('.xlsx') else 'Unknown'
    months.append(month)
    df = pd.read_excel(file_path)  # Read the Excel file into a DataFrame, skipping the first 2 rows
    
    # Lấy giá trị phí bảo hiềm tiền gửi trong file P&L
    val = df.iloc[126,3]/1000000000
    vals.append(val)
q4 = pd.DataFrame({
    'Month': months,
    'val': vals,
    'Khoản mục':'Chi phí bảo hiểm, bảo toàn tiền gửi'
})
q4['Month']=q4['Month'].astype(str)

# TÍNH TỔNG CHI PHÍ HOẠT ĐỘNG

In [50]:
# ghép các bảng chi phí
TH_df=pd.concat([q1,q2,q3,q4,q5])
TH_df=TH_df.drop_duplicates()

df1=TH_df[TH_df['Khoản mục']=="Tổng chi phí quản lý"]
df2=TH_df[TH_df['Khoản mục']=="Chi phí lương kinh doanh bảo hiểm"]
df3=TH_df[TH_df['Khoản mục']=="Chi phí bảo hiểm, bảo toàn tiền gửi"]
merged_df=pd.merge(df1,df2,on='Month').merge(df3,on='Month')
merged_df['new_val']=merged_df['val_x']+merged_df['val_y']+merged_df['val']
merged_df=merged_df[['Month','new_val']].rename(columns={'new_val':'val'})
merged_df['Khoản mục']='Tổng chi phí hoạt động'
total_TH_df=merged_df.copy()
# ghép bảng các chi phí và tổng chi phí hoạt động
new_TH_df=pd.concat([TH_df,total_TH_df])

# GỘP CÁC BẢNG DỮ LIỆU VÀ TÍNH SỐ LŨY KẾ

In [51]:
# Dataframe chứa chi phí kinh doanh bảo hiểm, chi phí CBNV có điều chỉnh và chi phí khác
df1=new_TH_df.copy()

# TÍNH SỐ LIỆU LŨY KẾ ĐẾN THÁNG BÁO CÁO
df2=new_TH_df.groupby('Khoản mục')['val'].sum().reset_index()
df2['Month']='Lũy Kế'

# File chi phí các tháng + tổng chi phí lũy kế các tháng
modified_TH_df=pd.concat([df1,df2])
# làm tròn số tới 2 chữ số 0
modified_TH_df['Thực hiện -']=modified_TH_df['val'].round(2)
modified_TH_df=modified_TH_df.drop(columns='val')

# SỐ LIỆU KẾ HOẠCH

In [52]:
file_path=r'D:\Bao Cao Chi Phí\info-ChiPhi.xlsx'
df=pd.read_excel(file_path,sheet_name='KH theo KM',skiprows=3)
df=df.iloc[0:29,4:]

melted_df = pd.melt(df, id_vars=['Unnamed: 4'], var_name='Month', value_name='Kế hoạch -')
melted_df=melted_df[melted_df['Month']!="Tổng hợp"].rename(columns={'Unnamed: 4':'Khoản mục'})
melted_df.Month=melted_df.Month.str.replace('Tháng ','').astype(int)
KH_df=melted_df.copy()

# KH 10 ngày lương tháng 12 năm 2023

In [53]:
# +50 tỷ vào chi phí cho CBNV của tháng 1
KH_df.loc[(KH_df.Month==1)&(KH_df['Khoản mục']=='Chi phí cho CBNV'),'Kế hoạch -']+=50
# làm cho tổng chi phí quản lý tháng 1 tăng 50 tỷ theo
KH_df.loc[(KH_df.Month==1)&(KH_df['Khoản mục']=='Tổng chi phí quản lý'),'Kế hoạch -']+=50

df1=KH_df[KH_df['Khoản mục']=="Tổng chi phí quản lý"]
df2=KH_df[KH_df['Khoản mục']=="Chi phí lương kinh doanh bảo hiểm"]
df3=KH_df[KH_df['Khoản mục']=="Chi phí bảo hiểm, bảo toàn tiền gửi"]
merged_df=pd.merge(df1,df2,on='Month').merge(df3,on='Month')
merged_df['val']=merged_df['Kế hoạch -_x']+merged_df['Kế hoạch -_y']+merged_df['Kế hoạch -']
merged_df['Khoản mục']='Tổng chi phí hoạt động'
total_KH_df=merged_df[['Khoản mục','Month','val']].rename(columns={'val':'Kế hoạch -'})

new_KH_df=pd.concat([KH_df,total_KH_df])

# lấy số kế hoạch của những tháng tương ứng với số tháng của số liệu thực hiện
df1=new_KH_df[new_KH_df.Month<=file_num]
# tỉnh tổng số kế hoạch lũy kế
df2=df1.groupby('Khoản mục')['Kế hoạch -'].sum().reset_index()
df2['Month']='Lũy Kế'
modified_KH_df=pd.concat([df1,df2])
# làm tròn số tới 2 chữ số 0
modified_KH_df['Kế hoạch -']=modified_KH_df['Kế hoạch -'].round(2)
modified_KH_df['Month']=modified_KH_df['Month'].astype(str)

# GHÉP SỐ LIỆU KẾ HOẠCH VÀ SỐ THỰC HIỆN

In [54]:
merged_df=pd.merge(modified_TH_df,modified_KH_df,on=['Khoản mục','Month'])
# tạo cột % thực hiện so với KH
merged_df['% so KH -']=(merged_df['Kế hoạch -']/merged_df['Thực hiện -']*100).round()
df=merged_df.copy()
# thay thế các giá trị inf vs 0
df.replace(np.inf, 0, inplace=True)
# chuyển cột phần trăm thành số nguyên và thêm kí hiệu %
df['% so KH -']=df['% so KH -'].astype(int).astype(str)+"%"

filtered_rows = df[df['Month'] != 'Lũy Kế']
filtered_rows['Month'] = 'Tháng ' + filtered_rows['Month'].astype(str)
df=pd.concat([filtered_rows,df[df['Month'] == 'Lũy Kế']])
df=df.drop_duplicates()

# Xoay dữ liệu thành chiều ngang với các tháng là các cột
# Example pivot operation, replace this with your actual pivot operation
pivot_df = df.pivot(index='Khoản mục', columns='Month', values=['Kế hoạch -', 'Thực hiện -', '% so KH -']).reset_index()

# Flatten the multi-level column index and convert integers to strings
pivot_df.columns = [' '.join(map(str, col)).strip() for col in pivot_df.columns.values]
# tạo danh sách các list KH, TH và %
KH_columns = [col for col in pivot_df.columns if "Kế hoạch" in col]
index_to_move = KH_columns.index('Kế hoạch - Lũy Kế')
KH_columns.pop(index_to_move)
KH_columns.append('Kế hoạch - Lũy Kế')

TH_columns = [col for col in pivot_df.columns if "Thực hiện" in col]
index_to_move = TH_columns.index('Thực hiện - Lũy Kế')
TH_columns.pop(index_to_move)
TH_columns.append('Thực hiện - Lũy Kế')

PCT_columns = [col for col in pivot_df.columns if "% so KH" in col]
index_to_move = PCT_columns.index('% so KH - Lũy Kế')
PCT_columns.pop(index_to_move)
PCT_columns.append('% so KH - Lũy Kế')

# sắp xếp các cột theo thứ tự thực hiện, kế hoạch, % so KH
combined_columns = [a for pair in zip(TH_columns, KH_columns, PCT_columns) for a in pair]

df2=pivot_df[combined_columns].round(2).fillna(0)
df1=pivot_df['Khoản mục']
q=pd.concat([df1, df2], axis=1)

# SẮP XẾP CÁC KHOẢN MỤC THEO THỨ TỰ
df=ChiPhiTaiKhoan_df.copy()
df=df[df.STT.notnull()]
merged_df=df[['STT','Loại chi phí (đ/c)']].rename(columns={'Loại chi phí (đ/c)':'Khoản mục'}).merge(q).drop_duplicates().sort_values(by='STT')
merged_df.STT=merged_df.STT.astype(int)
final_df=merged_df.copy()

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\4209430393.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rows['Month'] = 'Tháng ' + filtered_rows['Month'].astype(str)


In [55]:
# HOÀN THÀNH 1 SHEET KHOẢN MỤC - THÁNG
KhoanMuc_Thang_df=final_df.copy()
KhoanMuc_Thang_df.head()

,STT,Khoản mục,Thực hiện - Tháng 1,Kế hoạch - Tháng 1,% so KH - Tháng 1,Thực hiện - Tháng 2,Kế hoạch - Tháng 2,% so KH - Tháng 2,Thực hiện - Tháng 3,Kế hoạch - Tháng 3,...,% so KH - Tháng 5,Thực hiện - Tháng 6,Kế hoạch - Tháng 6,% so KH - Tháng 6,Thực hiện - Tháng 7,Kế hoạch - Tháng 7,% so KH - Tháng 7,Thực hiện - Lũy Kế,Kế hoạch - Lũy Kế,% so KH - Lũy Kế
0,1,Chi phí cho CBNV,164.61,327.14,199%,312.71,292.60,94%,305.54,350.22,...,105%,287.08,429.79,150%,312.85,295.14,94%,1947.17,2278.80,117%
1,2,Chi phí marketing,17.26,34.28,199%,14.53,34.28,236%,32.96,34.28,...,267%,57.48,34.28,60%,52.38,34.28,65%,225.10,239.94,107%
2,3,Chi phí hoạt động quản lý,52.83,101.61,192%,80.00,101.61,127%,89.30,101.61,...,101%,138.61,101.61,73%,108.11,101.61,94%,674.94,711.26,105%
3,4,Vật liệu giấy tờ in,1.36,5.41,398%,5.82,5.41,93%,5.08,5.41,...,167%,5.90,5.41,92%,4.61,5.41,117%,29.27,37.86,129%
4,5,Công tác phí,2.62,5.54,211%,4.70,5.54,118%,5.90,5.54,...,115%,5.31,5.54,104%,4.50,5.54,123%,32.89,38.79,118%


# 2. KHOẢN MỤC - QUÝ

In [56]:
# loại bỏ cột lũy kế theo tháng
df=modified_TH_df[modified_TH_df['Month']!="Lũy Kế"]
# thêm cột quý
df['Month']=df['Month'].astype(int)

# Define a function to map months to quarters
def map_month_to_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    else:
        return 4
# Apply the mapping function to create the 'Quarter' column
df['Quarter'] = df['Month'].apply(map_month_to_quarter)

df=df.groupby(['Khoản mục','Quarter'])['Thực hiện -'].sum().reset_index()
df['CumulativeSum']=df.groupby('Khoản mục')['Thực hiện -'].transform('cumsum')
df1=df[['Khoản mục','Quarter','Thực hiện -']]
df1['Quarter']="Quý "+df1['Quarter'].astype(str)
df2=df[['Khoản mục','Quarter','CumulativeSum']].rename(columns={'CumulativeSum':'Thực hiện -'})
df2['Quarter']="Lũy kế quý "+df2['Quarter'].astype(str)

# Số liệu thực hiện theo quý
modified_TH_df1=pd.concat([df1,df2])
modified_TH_df1.head()

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\3789286533.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month']=df['Month'].astype(int)
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\3789286533.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Quarter'] = df['Month'].apply(map_month_to_quarter)
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\3789286533.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,Khoản mục,Quarter,Thực hiện -
0,Bảo hiểm tài sản,Quý 1,4.04
1,Bảo hiểm tài sản,Quý 2,1.60
2,Bảo hiểm tài sản,Quý 3,0.60
3,"Bảo vệ tài sản, PCCC",Quý 1,42.40
4,"Bảo vệ tài sản, PCCC",Quý 2,45.85


In [57]:
# loại bỏ cột lũy kế theo tháng
df=modified_KH_df[modified_KH_df['Month']!="Lũy Kế"]
# thêm cột quý
df['Month']=df['Month'].astype(int)

# Define a function to map months to quarters
def map_month_to_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    else:
        return 4
# Apply the mapping function to create the 'Quarter' column
df['Quarter'] = df['Month'].apply(map_month_to_quarter)

df=df.groupby(['Khoản mục','Quarter'])['Kế hoạch -'].sum().reset_index()
df['CumulativeSum']=df.groupby('Khoản mục')['Kế hoạch -'].transform('cumsum')
modified_KH_df1=df.copy()

df=df.groupby(['Khoản mục','Quarter'])['Kế hoạch -'].sum().reset_index()
df['CumulativeSum']=df.groupby('Khoản mục')['Kế hoạch -'].transform('cumsum')
df1=df[['Khoản mục','Quarter','Kế hoạch -']]
df1['Quarter']="Quý "+df1['Quarter'].astype(str)
df2=df[['Khoản mục','Quarter','CumulativeSum']].rename(columns={'CumulativeSum':'Kế hoạch -'})
df2['Quarter']="Lũy kế quý "+df2['Quarter'].astype(str)

# số liệu kế hoạch theo quý
modified_KH_df1=pd.concat([df1,df2])
modified_KH_df1.head()

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\2363338210.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month']=df['Month'].astype(int)
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\2363338210.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Quarter'] = df['Month'].apply(map_month_to_quarter)
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\2363338210.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,Khoản mục,Quarter,Kế hoạch -
0,Bảo hiểm tài sản,Quý 1,2.22
1,Bảo hiểm tài sản,Quý 2,2.22
2,Bảo hiểm tài sản,Quý 3,0.74
3,"Bảo vệ tài sản, PCCC",Quý 1,43.35
4,"Bảo vệ tài sản, PCCC",Quý 2,43.35


# GHÉP SỐ LIỆU KẾ HOẠCH VÀ SỐ THỰC HIỆN

In [58]:
merged_df=pd.merge(modified_TH_df1,modified_KH_df1,on=['Khoản mục','Quarter'])
# tạo cột % thực hiện so với KH
merged_df['% so KH -']=(merged_df['Kế hoạch -']/merged_df['Thực hiện -']*100).round()
df=merged_df.copy()
# thay thế các giá trị inf vs 0
df.replace(np.inf, 0, inplace=True)
# chuyển cột phần trăm thành số nguyên và thêm kí hiệu %
df['% so KH -']=df['% so KH -'].astype(int).astype(str)+"%"
df=df.sort_values(by='Quarter')


In [59]:
# Xoay dữ liệu thành chiều ngang với 
# Example pivot operation, replace this with your actual pivot operation
pivot_df = df.pivot(index='Khoản mục', columns='Quarter', values=['Kế hoạch -', 'Thực hiện -', '% so KH -']).reset_index()
# Flatten the multi-level column index and convert integers to strings
pivot_df.columns = [' '.join(map(str, col)).strip() for col in pivot_df.columns.values]
pivot_df.columns

Index(['Khoản mục', 'Kế hoạch - Lũy kế quý 1', 'Kế hoạch - Lũy kế quý 2',
       'Kế hoạch - Lũy kế quý 3', 'Kế hoạch - Quý 1', 'Kế hoạch - Quý 2',
       'Kế hoạch - Quý 3', 'Thực hiện - Lũy kế quý 1',
       'Thực hiện - Lũy kế quý 2', 'Thực hiện - Lũy kế quý 3',
       'Thực hiện - Quý 1', 'Thực hiện - Quý 2', 'Thực hiện - Quý 3',
       '% so KH - Lũy kế quý 1', '% so KH - Lũy kế quý 2',
       '% so KH - Lũy kế quý 3', '% so KH - Quý 1', '% so KH - Quý 2',
       '% so KH - Quý 3'],
      dtype='object')

In [60]:
KH_columns1 = [col for col in pivot_df.columns if "Kế hoạch - Lũy" in col]
KH_columns2 = [col for col in pivot_df.columns if "Kế hoạch - Quý" in col]
KH_columns=KH_columns2+KH_columns1


TH_columns1 = [col for col in pivot_df.columns if "Thực hiện - Lũy" in col]
TH_columns2 = [col for col in pivot_df.columns if "Thực hiện - Quý" in col]
TH_columns=TH_columns2+TH_columns1


PCT_columns1 = [col for col in pivot_df.columns if "% so KH" in col]
PCT_columns2 = [col for col in pivot_df.columns if "% so KH" in col]
PCT_columns=PCT_columns2+PCT_columns1

# sắp xếp các cột theo thứ tự thực hiện, kế hoạch, % so KH
combined_columns = [a for pair in zip(TH_columns, KH_columns, PCT_columns) for a in pair]
df2=pivot_df[combined_columns].round(2).fillna(0)
df1=pivot_df['Khoản mục']
q=pd.concat([df1, df2], axis=1)
q

,Khoản mục,Thực hiện - Quý 1,Kế hoạch - Quý 1,% so KH - Lũy kế quý 1,Thực hiện - Quý 2,Kế hoạch - Quý 2,% so KH - Lũy kế quý 2,Thực hiện - Quý 3,Kế hoạch - Quý 3,% so KH - Lũy kế quý 3,Thực hiện - Lũy kế quý 1,Kế hoạch - Lũy kế quý 1,% so KH - Quý 1,Thực hiện - Lũy kế quý 2,Kế hoạch - Lũy kế quý 2,% so KH - Quý 2,Thực hiện - Lũy kế quý 3,Kế hoạch - Lũy kế quý 3,% so KH - Quý 3
0,Bảo hiểm tài sản,4.04,2.22,55%,1.60,2.22,79%,0.60,0.74,83%,4.04,2.22,55%,5.64,4.44,139%,6.24,5.18,123%
1,"Bảo vệ tài sản, PCCC",42.40,43.35,102%,45.85,43.35,98%,15.15,14.45,98%,42.40,43.35,102%,88.25,86.70,95%,103.40,101.15,95%
2,Chi khác,12.35,29.46,239%,34.31,29.46,126%,9.40,9.82,123%,12.35,29.46,239%,46.66,58.92,86%,56.06,68.74,104%
3,Chi phí HĐQT và BKS,10.54,5.64,54%,7.25,5.64,63%,3.86,1.88,61%,10.54,5.64,54%,17.79,11.28,78%,21.65,13.16,49%
4,"Chi phí bảo hiểm, bảo toàn tiền gửi",50.67,57.99,114%,61.05,57.99,104%,26.44,19.33,98%,50.67,57.99,114%,111.72,115.98,95%,138.16,135.31,73%
5,Chi phí cho CBNV,782.86,969.96,124%,851.46,1013.69,121%,312.85,295.14,117%,782.86,969.96,124%,1634.32,1983.65,119%,1947.17,2278.79,94%
6,Chi phí hoạt động quản lý,222.13,304.83,137%,344.70,304.83,108%,108.11,101.61,105%,222.13,304.83,137%,566.83,609.66,88%,674.94,711.27,94%
7,Chi phí lương kinh doanh bảo hiểm,60.55,160.11,264%,109.95,160.11,188%,7.13,53.37,210%,60.55,160.11,264%,170.50,320.22,146%,177.63,373.59,749%
8,Chi phí marketing,64.75,102.84,159%,107.97,102.84,119%,52.38,34.28,107%,64.75,102.84,159%,172.72,205.68,95%,225.10,239.96,65%
9,Chi phí thu hồi nợ,0.10,1.74,1740%,0.37,1.74,740%,0.16,0.58,644%,0.10,1.74,1740%,0.47,3.48,470%,0.63,4.06,362%


In [61]:
# SẮP XẾP CÁC KHOẢN MỤC THEO THỨ TỰ
df=ChiPhiTaiKhoan_df.copy()
df=df[df.STT.notnull()]
merged_df=df[['STT','Loại chi phí (đ/c)']].rename(columns={'Loại chi phí (đ/c)':'Khoản mục'}).merge(q).drop_duplicates().sort_values(by='STT')
merged_df.STT=merged_df.STT.astype(int)
final_df=merged_df.copy()


In [62]:
# HOÀN THÀNH 1 SHEET KHOẢN MỤC - QUÝ
KhoanMuc_Quy_df=merged_df.copy()

# 3. DATA ĐỒ THỊ THEO CHI PHÍ CỦA CÁC KHOẢN MỤC THEO THÁNG

In [63]:
chart_df=modified_TH_df.pivot(index='Khoản mục', columns='Month', values='Thực hiện -').reset_index()

# lọc ra các cột chi phí cần dựng đồ thị
list=['Chi phí cho CBNV','Chi phí marketing','Chi phí hoạt động quản lý','Chi phí về tài sản',
      'Chi phí HĐQT và BKS','Chi phí lương kinh doanh bảo hiểm','Chi phí bảo hiểm, bảo toàn tiền gửi'
     ]
df=chart_df[chart_df['Khoản mục'].isin(list)]
# làm tròn giá trị cho bảng dataframe
df=df.round()
df.iloc[:,1:]=df.iloc[:,1:].astype(int)

# TẠO CỘT CHI PHÍ KHÁC+BHTG = CHI PHÍ HDQT VÀ BKS + CHI PHÍ BẢO HIỂM, BÀO TOÀN TIỀN GỬI
# xoay dataframe thành chiều dọc
transposed_df=df.T
transposed_df.columns=transposed_df.iloc[0,:]
filtered_df=transposed_df.iloc[1:,:]
# tạo cột chi phí khác + bhtg
filtered_df['Chi phí khác+BHTG']=filtered_df['Chi phí HĐQT và BKS']+filtered_df['Chi phí bảo hiểm, bảo toàn tiền gửi']
filtered_df['Tổng']=filtered_df['Chi phí cho CBNV'
                               ]+filtered_df['Chi phí lương kinh doanh bảo hiểm'
                                            ]+filtered_df['Chi phí marketing'
                                                         ]+filtered_df['Chi phí hoạt động quản lý'
                                                                      ]+filtered_df['Chi phí về tài sản'
                                                                                   ]+filtered_df['Chi phí khác+BHTG']


C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\2480224271.py:10: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,1:]=df.iloc[:,1:].astype(int)
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\2480224271.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Chi phí khác+BHTG']=filtered_df['Chi phí HĐQT và BKS']+filtered_df['Chi phí bảo hiểm, bảo toàn tiền gửi']
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_21108\2480224271.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [64]:
# sắp xếp các cột theo thứ tự mới
ordered_cols=['Chi phí cho CBNV', 'Chi phí lương kinh doanh bảo hiểm', 'Chi phí marketing', 'Chi phí hoạt động quản lý',
                'Chi phí về tài sản','Chi phí khác+BHTG','Tổng']
# xoay dataframe lại thành chiều ngang
new_chart_df=filtered_df[ordered_cols].T.reset_index()
KhoanMuc_Thang_chart_df=new_chart_df.copy()
KhoanMuc_Thang_chart_df.head()

Month,Khoản mục,1,2,3,4,5,6,7,Lũy Kế
0,Chi phí cho CBNV,165,313,306,287,278,287,313,1947
1,Chi phí lương kinh doanh bảo hiểm,58,2,0,71,30,9,7,178
2,Chi phí marketing,17,15,33,38,13,57,52,225
3,Chi phí hoạt động quản lý,53,80,89,106,101,139,108,675
4,Chi phí về tài sản,61,72,77,71,75,72,76,502


In [65]:
# ĐỒ THỊ THEO CƠ CẤU PHẦN TRĂM:
df=filtered_df.copy()
df['CP cho CBNV']=(df['Chi phí cho CBNV']/df['Tổng'])
df['CP hoạt động quản lý']=(df['Chi phí hoạt động quản lý']/df['Tổng'])
df['CP LKD bảo hiểm']=(df['Chi phí lương kinh doanh bảo hiểm']/df['Tổng'])
df['CP Marketing']=(df['Chi phí marketing']/df['Tổng'])
df['CP về tài sản']=(df['Chi phí về tài sản']/df['Tổng'])
df['CP khác+BHTG']=(df['Chi phí khác+BHTG']/df['Tổng'])
df['Tổng CP']=(df['Tổng']/df['Tổng'])

order_cols=['CP cho CBNV', 'CP LKD bảo hiểm', 'CP Marketing', 'CP hoạt động quản lý',
                'CP về tài sản','CP khác+BHTG','Tổng CP']
KhoanMuc_Thang_chart_in_pct_df=df[order_cols].T.reset_index()
KhoanMuc_Thang_chart_in_pct_df.head()

Month,Khoản mục,1,2,3,4,5,6,7,Lũy Kế
0,CP cho CBNV,0.441176,0.621032,0.583969,0.482353,0.535645,0.488927,0.53413,0.528072
1,CP LKD bảo hiểm,0.15508,0.003968,0.0,0.119328,0.057803,0.015332,0.011945,0.048278
2,CP Marketing,0.045455,0.029762,0.062977,0.063866,0.025048,0.097104,0.088737,0.061025
3,CP hoạt động quản lý,0.141711,0.15873,0.169847,0.178151,0.194605,0.236797,0.1843,0.183076
4,CP về tài sản,0.163102,0.142857,0.146947,0.119328,0.144509,0.122658,0.129693,0.136154


In [66]:
file_path=r'D:\Bao Cao Chi Phí\info-ChiPhi.xlsx'
df=pd.read_excel(file_path,sheet_name='KH theo KM',skiprows=3).iloc[0:30,4:].rename(columns={'Unnamed: 4':'Khoản mục'})

In [67]:
melted_df = pd.melt(df, id_vars=['Khoản mục'], var_name='Month', value_name='Tar')
melted_df=melted_df[melted_df['Month']!='Tổng hợp']
melted_df['Month']=melted_df['Month'].str.replace('Tháng ','')
melted_df['Month']=melted_df['Month'].astype(int)
# lọc ra các chi phí trong file kế hoạch
check_list =['Chi phí cho CBNV','Chi phí lương kinh doanh bảo hiểm','Chi phí marketing',
       'Chi phí hoạt động quản lý','Chi phí về tài sản', 'Chi phí HĐQT và BKS',
      'Chi phí bảo hiểm, bảo toàn tiền gửi']

filtered_df=melted_df[melted_df['Khoản mục'].isin(check_list)]
# đổi tên các khoản mục lọc ra của file kế hoạch để giống file thực hiện
filtered_df.loc[filtered_df['Khoản mục']=='Chi phí cho CBNV','Khoản mục']='Chi phí cho CBNV'
filtered_df.loc[filtered_df['Khoản mục']=='Chi phí lương kinh doanh bảo hiểm','Khoản mục']='Chi phí lương kinh doanh bảo hiểm'
filtered_df.loc[filtered_df['Khoản mục']=='Chi phí marketing','Khoản mục']='Chi phí marketing'
filtered_df.loc[filtered_df['Khoản mục']=='Chi phí hoạt động quản lý','Khoản mục']='Chi phí hoạt động quản lý'
filtered_df.loc[filtered_df['Khoản mục']=='Chi phí về tài sản','Khoản mục']='Chi phí về tài sản'
# các chi phí # chi phí bảo hiểm, bảo toàn tiền gửi và chi phí HDQT và BKS
q1=filtered_df[~filtered_df['Khoản mục'].isin(['Chi phí bảo hiểm, bảo toàn tiền gửi','Chi phí HĐQT và BKS'])]
# Chi phí khác+BHTG = Chi phí HĐQT và BKS + Chi phí bảo hiểm, bảo toàn tiền gửi
df1=filtered_df[filtered_df['Khoản mục']=='Chi phí bảo hiểm, bảo toàn tiền gửi']
df2=filtered_df[filtered_df['Khoản mục']=='Chi phí HĐQT và BKS']
merged_df=pd.merge(df1,df2,on='Month')
merged_df['Tar']=merged_df['Tar_x']+merged_df['Tar_y']
merged_df['Khoản mục']='CP khác+BHTG'
q2=merged_df[['Khoản mục','Month','Tar']]
# file kế hoạch cuối cùng để so sánh
KH_df=pd.concat([q1,q2])
# tính số lũy kế của kế hoạch tháng đặt ra
KH_df=KH_df[KH_df['Month']<=file_num].groupby('Khoản mục').Tar.sum().reset_index()
# file data đồ thị so sánh các khoản mục
KhoanMuc_Thang_So_KH_df=new_chart_df[['Khoản mục','Lũy Kế']].merge(KH_df,on='Khoản mục').rename(columns={'Lũy Kế':f'TH LK Tháng {file_num}','Tar':f'KH LK Tháng {file_num}'})
KhoanMuc_Thang_So_KH_df['KH LK Tháng 7']=KhoanMuc_Thang_So_KH_df['KH LK Tháng 7'].astype(int)
KhoanMuc_Thang_So_KH_df.head()

,Khoản mục,TH LK Tháng 7,KH LK Tháng 7
0,Chi phí cho CBNV,1947,2228
1,Chi phí lương kinh doanh bảo hiểm,178,373
2,Chi phí marketing,225,239
3,Chi phí hoạt động quản lý,675,711
4,Chi phí về tài sản,502,514


# PHIÊN BẢN TIẾNG ANH 3 ĐỒ THỊ TRÊN

In [68]:
# phiên bản tiếng Anh cho
df1=KhoanMuc_Thang_chart_df.copy()

replacement_dict1 ={'Chi phí cho CBNV':'Personnel expenses','Chi phí lương kinh doanh bảo hiểm':'Insurance KPI bonus',
 'Chi phí marketing':'Marketing expenses','Chi phí hoạt động quản lý':'Administrative expenses',
 'Chi phí khác+BHTG':"Other operating expensive & Insurance fee for customer deposit",
 'Chi phí về tài sản':'Fixed asset expenses','Tổng':'OPEX'
}
# Replace values in the 'Khoản mục' column
df1['Khoản mục'] = df1['Khoản mục'].replace(replacement_dict1)

df2=KhoanMuc_Thang_chart_in_pct_df.copy()
replacement_dict2 ={'CP cho CBNV':'Personnel expenses','CP LKD bảo hiểm':'Insurance KPI bonus',
 'CP marketing':'Marketing expenses','CP hoạt động quản lý':'Administrative expenses',
 'CP khác+BHTG':"Other operating expensive & Insurance fee for customer deposit",
 'CP về tài sản':'Fixed asset expenses','Tổng CP':'OPEX'
}
df2['Khoản mục'] = df2['Khoản mục'].replace(replacement_dict2)
KhoanMuc_Thang_chart_df_Eng=df1.copy()
KhoanMuc_Thang_chart_in_pct_df_Eng=df2.copy()

In [69]:
def replace_column_names(df):
    original_list = df.columns
    new_list = ['Item' if item == 'Khoản mục' else item.replace('Quý ', 'Quarter ') for item in original_list]
    df.columns = new_list

# Replace column names for KhoanMuc_Quy_chart_df_Eng
replace_column_names(KhoanMuc_Thang_chart_df_Eng)

# Replace column names for KhoanMuc_Quy_chart_in_pct_df_Eng
replace_column_names(KhoanMuc_Thang_chart_in_pct_df_Eng)

In [86]:
df3=KhoanMuc_Thang_So_KH_df.copy()
df3['Khoản mục']=df3['Khoản mục'].replace(replacement_dict1)
KhoanMuc_Thang_So_KH_df_Eng=df3.copy()

In [87]:
columns = KhoanMuc_Thang_So_KH_df_Eng.columns
columns = [column.replace('Khoản mục', 'Item') for column in columns]
columns = [column.replace('Khoản mục', 'Item') for column in columns]

number_to_month = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}
# Convert the number to a month and save it into a new variable
month = number_to_month.get(file_num)

columns[1]=f'Actual expensive - {month}'
columns[2]=f'Budget expensive - {month}'

In [88]:
KhoanMuc_Thang_So_KH_df_Eng.columns=columns
KhoanMuc_Thang_So_KH_df_Eng

,Item,Actual expensive - July,Budget expensive - July
0,Personnel expenses,1947,2228
1,Insurance KPI bonus,178,373
2,Marketing expenses,225,239
3,Administrative expenses,675,711
4,Fixed asset expenses,502,514


# 4. DATA ĐỒ THỊ THEO KHOẢN MỤC THEO QUÝ

In [32]:
# lọc ra các cột chi phí cần dựng đồ thị
list=['Chi phí cho CBNV','Chi phí marketing','Chi phí hoạt động quản lý','Chi phí về tài sản',
      'Chi phí HĐQT và BKS','Chi phí lương kinh doanh bảo hiểm','Chi phí bảo hiểm, bảo toàn tiền gửi'
     ]
df=modified_TH_df1[modified_TH_df1['Khoản mục'].isin(list)]

# loại bỏ các giá trị về lũy kế
df=df[~df['Quarter'].str.contains('Lũy kế')]
df['Thực hiện -']=df['Thực hiện -'].astype(int)
pivot_df=df.pivot(index='Quarter', columns='Khoản mục', values='Thực hiện -')

# tạo cột chi phí khác + bhtg
pivot_df['Chi phí khác+BHTG']=pivot_df['Chi phí HĐQT và BKS']+pivot_df['Chi phí bảo hiểm, bảo toàn tiền gửi']
pivot_df['Tổng']=pivot_df['Chi phí cho CBNV'
                               ]+pivot_df['Chi phí lương kinh doanh bảo hiểm'
                                            ]+pivot_df['Chi phí marketing'
                                                         ]+pivot_df['Chi phí hoạt động quản lý'
                                                                      ]+pivot_df['Chi phí về tài sản'
                                                                                   ]+pivot_df['Chi phí khác+BHTG']

# sắp xếp các cột theo thứ tự mới
ordered_cols=['Chi phí cho CBNV', 'Chi phí lương kinh doanh bảo hiểm', 'Chi phí marketing', 'Chi phí hoạt động quản lý',
                'Chi phí về tài sản','Chi phí khác+BHTG','Tổng']
# xoay dataframe lại thành chiều ngang
new_chart_df=pivot_df[ordered_cols].T.reset_index()
KhoanMuc_Quy_chart_df=new_chart_df.copy()
KhoanMuc_Quy_chart_df

Quarter,Khoản mục,Quý 1,Quý 2,Quý 3
0,Chi phí cho CBNV,782,851,312
1,Chi phí lương kinh doanh bảo hiểm,60,109,7
2,Chi phí marketing,64,107,52
3,Chi phí hoạt động quản lý,222,344,108
4,Chi phí về tài sản,209,217,75
5,Chi phí khác+BHTG,60,68,29
6,Tổng,1397,1696,583


In [33]:
# ĐỒ THỊ THEO CƠ CẤU PHẦN TRĂM:
df=pivot_df.copy()
df['CP cho CBNV']=(df['Chi phí cho CBNV']/df['Tổng'])
df['CP hoạt động quản lý']=(df['Chi phí hoạt động quản lý']/df['Tổng'])
df['CP LKD bảo hiểm']=(df['Chi phí lương kinh doanh bảo hiểm']/df['Tổng'])
df['CP Marketing']=(df['Chi phí marketing']/df['Tổng'])
df['CP về tài sản']=(df['Chi phí về tài sản']/df['Tổng'])
df['CP khác+BHTG']=(df['Chi phí khác+BHTG']/df['Tổng'])
df['Tổng CP']=(df['Tổng']/df['Tổng'])

order_cols=['CP cho CBNV', 'CP LKD bảo hiểm', 'CP Marketing', 'CP hoạt động quản lý',
                'CP về tài sản','CP khác+BHTG','Tổng CP']
pct_chart_df1=df[order_cols].T.reset_index()
KhoanMuc_Quy_chart_in_pct_df=pct_chart_df1.copy()
KhoanMuc_Quy_chart_in_pct_df

Quarter,Khoản mục,Quý 1,Quý 2,Quý 3
0,CP cho CBNV,0.559771,0.501769,0.535163
1,CP LKD bảo hiểm,0.042949,0.064269,0.012007
2,CP Marketing,0.045812,0.063090,0.089194
3,CP hoạt động quản lý,0.158912,0.202830,0.185249
4,CP về tài sản,0.149606,0.127948,0.128645
5,CP khác+BHTG,0.042949,0.040094,0.049743
6,Tổng CP,1.000000,1.000000,1.000000


In [34]:
# phiên bản tiếng Anh cho
df1=KhoanMuc_Quy_chart_df.copy()

replacement_dict ={'Chi phí cho CBNV':'Personnel expenses','Chi phí lương kinh doanh bảo hiểm':'Insurance KPI bonus',
 'Chi phí marketing':'Marketing expenses','Chi phí hoạt động quản lý':'Administrative expenses',
 'Chi phí khác+BHTG':"Other operating expensive & Insurance fee for customer deposit",
 'Chi phí về tài sản':'Fixed asset expenses','Tổng':'OPEX'
}
# Replace values in the 'Khoản mục' column
df1['Khoản mục'] = df1['Khoản mục'].replace(replacement_dict)

df2=KhoanMuc_Quy_chart_in_pct_df.copy()
replacement_dict ={'CP cho CBNV':'Personnel expenses','CP LKD bảo hiểm':'Insurance KPI bonus',
 'CP marketing':'Marketing expenses','CP hoạt động quản lý':'Administrative expenses',
 'CP khác+BHTG':"Other operating expensive & Insurance fee for customer deposit",
 'CP về tài sản':'Fixed asset expenses','Tổng CP':'OPEX'
}
df2['Khoản mục'] = df2['Khoản mục'].replace(replacement_dict)
KhoanMuc_Quy_chart_df_Eng=df1.copy()
KhoanMuc_Quy_chart_in_pct_df_Eng=df2.copy()

In [35]:
# Replace column names for KhoanMuc_Quy_chart_df_Eng
replace_column_names(KhoanMuc_Quy_chart_df_Eng)

# Replace column names for KhoanMuc_Quy_chart_in_pct_df_Eng
replace_column_names(KhoanMuc_Quy_chart_in_pct_df_Eng)


# 4. KHỐI - THÁNG

In [36]:
df=bcqt19_df.copy()
# Find the position of "_" in each value of the "column_name" column
df['underscore_position'] = df['var'].str.find('_')
# Create a new column "substring_column" by extracting substrings based on "underscore_position"
df['category'] = df.apply(lambda row: row['var'][:row['underscore_position']] , axis=1)
df['Month'] = df.apply(lambda row: row['var'][row['underscore_position']+1:] , axis=1)
df=df.groupby(['code','Month']).val.sum().reset_index()
df['val']=df['val']/1000000000
data=df.copy()


# CHI PHÍ TẠI DVKD 

In [37]:
# lọc ra các chi phí của đơn vị kinh doanh
df1=data[data['code'].str.startswith('ĐVKD')]
# đổi lại mã của các đơn vị này thành DVKD
df1['Mã']='DVKD'
# Tỉnh tổng chi phí của tất cả các DVKD vào 1 mã DVKD
df1=df1[['Month','Mã','val']].groupby(['Month','Mã']).val.sum().reset_index()

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\3109999161.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Mã']='DVKD'


# TÁCH LƯƠNG KINH DOANH BẢO HIỂM RA KHỎI CHI PHÍ CỦA CỦA DVKD

In [38]:
file_path=r'D:\Bao Cao Chi Phí\N2023 - LƯƠNG KD - PC.xlsx'
df=pd.read_excel(file_path,sheet_name='N2023 Lương TH',skiprows=1)
# TẠO DANH SÁCH CÁC THÁNG
month_list=[]
for i in range(1,13):
    month_list.append(i)

# TẠO DANH SÁCH CHI PHÍ KD BẢO HIỂM
val_list=[]
for i in df.iloc[1,4:20][1:].values:
    val_list.append(i)
del val_list[13]
del val_list[12]
# Số liệu tháng 4 bằng element 
sum_value = val_list[2]+val_list[3]
# Remove elements at positions 1 and 2
del val_list[2]
del val_list[2]

# Số liệu tháng 3 = 0
new_value=0
val_list.insert(2, new_value)
# Số liệu tháng 4 mới
val_list.insert(3, sum_value)

# Creating a DataFrame
two_list = {'Month': month_list, 'val': val_list}
KDBH_df = pd.DataFrame(two_list)
df=KDBH_df.copy()
df['val']=df['val']/1e9
df['Month']=df['Month'].astype(str)
# thêm cột khoản mục Chi phí cho CBNV để nối bảng
df['Khoản mục']='Chi phí cho CBNV'
LuongKDBH_df=df.copy()

In [39]:
# ghép bảng chi phí DVKD vs bảng chi phí lương kd bảo hiểm
merged_df=df1.merge(df,on='Month')
merged_df['val']=(merged_df['val_x']-merged_df['val_y']).round(2)
modified_df1=merged_df[['Month','Mã','val']]

LuongKDBH_df['Mã']='Chi phí lương kinh doanh bảo hiểm'
modified_KDBH_df=LuongKDBH_df.drop(columns='Khoản mục')
modified_KDBH_df=modified_KDBH_df[modified_KDBH_df['Month'].astype(int)<=file_num]

# CHI PHÍ TẠI HO

In [40]:
# chi phí tại Hội SỞ
df2=data[~data['code'].str.startswith('ĐVKD')]

# lấy mapping Code vs dept
file_path = r'D:\Bao Cao Chi Phí\info-ChiPhi.xlsx'

df = pd.read_excel(file_path, sheet_name="CODE_MAP1") \
       .query('dept.notnull() and code.str.startswith("HO")')[['code', 'dept']]

df2=df2.merge(df,on='code',how='outer').groupby(['Month','dept']).val.sum().reset_index().rename(columns={'dept':'Mã'})

# kết hợp chi phí tại DVKD, chi phí tại hội sở và lương kinh doanh bảo hiểm
concatenated_df=pd.concat([modified_df1,df2,modified_KDBH_df])
concatenated_df['Month']=concatenated_df['Month'].astype(int)

# CỘNG THÊM CHI PHÍ CỘNG TAY Ở NGOÀI CHO VÀO CÁC KHỐI Ở HO

In [41]:
def process_excel_file(file_path, sheet_name, month):
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=4)
    categories = ['THUẾ MB, GTGT, KHNH, TNDN', 'CHI PHÍ DỰ PHÒNG GIẢM GIÁ VÀNG, HÀNG TỒN KHO…']
    filtered_df = df[df['HEADING_DESC'].isin(categories)]
    filtered_df.iloc[:, 2:-2] = filtered_df.iloc[:, 2:-2] / 1000000000

    codes = ['HO_CHUNG', 'HO_200', 'HO_100', 'HO_300', 'HO_350', 'HO_800', 'HO_400', 'HO_668']
    vals = [
        filtered_df['PS_BANKING_BOOK'].sum() + filtered_df['CAPITAL BOOK'].sum(),
        filtered_df['PS_CA_NHAN'].sum(),
        filtered_df['PS_DOANH_NGHIEP'].sum(),
        filtered_df['FX_SALE'].sum() + filtered_df['PS_TRADING_BOOK'].sum(),
        filtered_df['PHÒNG ĐẦU TƯ'].sum(),
        filtered_df['PS_DOANH_NGHIEP_LON'].sum(),
        filtered_df['TRUNG TÂM THẺ'].sum(),
        filtered_df['MUADEE'].sum()
    ]

    return pd.DataFrame({'Mã': codes, 'val': vals, 'Month': month})

results = []

# Process multiple Excel files in a loop
for i in range(1, 8):
    file_path = f'D:\\Bao Cao LNK\\LNK\\LNK {i}.xlsx'
    sheet_name = f'LN T{i} (ĐC DV các khối)'
    result = process_excel_file(file_path, sheet_name, i)
    results.append(result)

# Concatenate the results into a single DataFrame
final_result = pd.concat(results, ignore_index=True)
final_result

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\793206557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.iloc[:, 2:-2] = filtered_df.iloc[:, 2:-2] / 1000000000
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\793206557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.iloc[:, 2:-2] = filtered_df.iloc[:, 2:-2] / 1000000000
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\793206557.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

,Mã,val,Month
0,HO_CHUNG,0.000800,1
1,HO_200,0.271200,1
2,HO_100,0.075200,1
3,HO_300,0.000000,1
4,HO_350,0.000000,1
5,HO_800,0.000800,1
6,HO_400,0.000000,1
7,HO_668,0.000000,1
8,HO_CHUNG,0.000000,2
9,HO_200,0.000000,2


In [42]:
def process_excel_file(file_path, sheet_name, month):
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=4)
    categories = ['THUẾ MB, GTGT, KHNH, TNDN', 'CHI PHÍ DỰ PHÒNG GIẢM GIÁ VÀNG, HÀNG TỒN KHO…']
    filtered_df = df[df['HEADING_DESC'].isin(categories)]
    filtered_df.iloc[:, 2:-2] = filtered_df.iloc[:, 2:-2] / 1000000000

    codes = ['HO_CHUNG', 'HO_200', 'HO_100', 'HO_300', 'HO_350', 'HO_800', 'HO_400', 'HO_668']
    vals = [
        filtered_df['PS_BANKING_BOOK'].sum() + filtered_df['CAPITAL BOOK'].sum(),
        filtered_df['PS_CA_NHAN'].sum(),
        filtered_df['PS_DOANH_NGHIEP'].sum(),
        filtered_df['FX_SALE'].sum() + filtered_df['PS_TRADING_BOOK'].sum(),
        filtered_df['PHÒNG ĐẦU TƯ'].sum(),
        filtered_df['PS_DOANH_NGHIEP_LON'].sum(),
        filtered_df['TRUNG TÂM THẺ'].sum(),
        filtered_df['MUADEE'].sum()
    ]

    return pd.DataFrame({'Mã': codes, 'val': vals, 'Month': month})

results = []

# Process multiple Excel files in a loop
for i in range(1, 8):
    file_path = f'D:\\Bao Cao LNK\\LNK\\LNK {i}.xlsx'
    sheet_name = f'LN T{i} (ĐC DV các khối)'
    result = process_excel_file(file_path, sheet_name, i)
    results.append(result)

# Concatenate the results into a single DataFrame
final_result = pd.concat(results, ignore_index=True)

# cộng vào số liệu của file điều chỉnh
merged_df=concatenated_df.merge(final_result,on=['Month','Mã'],how='outer')
merged_df['val_y']=merged_df['val_y'].fillna(0)
merged_df['val']=(merged_df['val_x']+merged_df['val_y'])
# lấy các cột cần thiết
merged_df=merged_df[['Month','Mã','val']]

# số liệu chi phí thực hiện của các tháng
TH_df=merged_df.copy()

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\1322370560.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.iloc[:, 2:-2] = filtered_df.iloc[:, 2:-2] / 1000000000
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\1322370560.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.iloc[:, 2:-2] = filtered_df.iloc[:, 2:-2] / 1000000000
C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\1322370560.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

# TỔNG CHI PHÍ QUẢN LÝ

In [43]:
# TỔNG CHI PHÍ QUẢN LÝ = TẤT CẢ CÁC CHI PHÍ CỦA CÁC KHỐI VÀ DVKD (TRỪ CHI PHÍ LƯƠNG KDBH)
TongChiPhi_df=TH_df[TH_df['Mã']!='Chi phí lương kinh doanh bảo hiểm'].groupby('Month').val.sum().reset_index()
TongChiPhi_df['Mã']='Tổng chi phí quản lý'


# CHÍ PHÍ BẢO HIỂM BẢO TOÀN TIỀN GỬI

In [44]:
import pandas as pd

folder_path = r'D:\Bao Cao LNK\LNK\\'  # Specify the common folder path

# Initialize an empty list to store the results
vals = []
months = []

# Specify the sheet names
sheet_names = ['LN T1 (ĐC DV các khối)', 'LN T2 (ĐC DV các khối)', 'LN T3 (ĐC DV các khối)',
               'LN T4 (ĐC DV các khối)', 'LN T5 (ĐC DV các khối)', 'LN T6 (ĐC DV các khối)',
              'LN T7 (ĐC DV các khối)']

# Iterate through the sheet names and create file paths
for i, sheet_name in enumerate(sheet_names):
    file_path = f'{folder_path}LNK {i + 1}.xlsx'
    
    # Read the Excel file
    df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=4)
    
    # Calculate the desired value
    val = (df[df['HEADING_DESC'] == 'CHI PHÍ BẢO HIỂM, BẨO TOÀN TIỂN GỬI']['PS_TH'].sum()/1000000000)
    months.append(i+1)
    
    # Append the value to the list
    vals.append(val)
    result_df = pd.DataFrame({'Month': months, 'val': vals})
result_df['Mã']='Chi phí bảo hiểm, bảo toàn tiền gửi'
BHTG_df=result_df.copy()


# TỔNG CHI PHÍ HOẠT ĐỘNG

In [45]:
# TỔNG CHI PHÍ HOẠT ĐỘNG = 
df=pd.concat([TongChiPhi_df,BHTG_df,TH_df[TH_df['Mã']=='Chi phí lương kinh doanh bảo hiểm']])
TongChiPhiHoatDong_df=df.groupby('Month').val.sum().reset_index()
TongChiPhiHoatDong_df['Mã']='Tổng chi phí hoạt động'

# BẢNG CHI PHÍ GỘP (CP CÁC KHỐI, CP DVKD, LƯƠNG KDBH, TỔNG CP QUẢN LÝ & BH TIỀN GỬI)

In [46]:
# BẢNG TỔNG HỢP CÁC CHI PHÍ
new_TH_df=pd.concat([TH_df,TongChiPhi_df,BHTG_df,TongChiPhiHoatDong_df])
new_TH_df['val']=new_TH_df['val'].round(2)

# TÍNH GIÁ TRỊ LŨY KẾ

In [47]:
# số liệu tổng chi phí lũy kế của các tháng
LuyKe_df=new_TH_df.groupby(['Mã']).val.sum().reset_index()
LuyKe_df['Month']='Lũy kế'

# BẢNG CHI PHÍ CUỐI CÙNG
final_TH_df=pd.concat([new_TH_df,LuyKe_df])
# số thực hiện
final_TH_df.Month=final_TH_df.Month.astype(str)
df1=final_TH_df.copy()

# SỐ LIỆU KẾ HOẠCH

In [48]:
file_path=r'D:\Bao Cao Chi Phí\info-ChiPhi.xlsx'
df=pd.read_excel(file_path,sheet_name='KH theo Khoi',skiprows=2)
# xử lý cột mã để nối 2 bảng
df['Mã1']=df['Mã']
df.loc[df['Mã'].isnull(),'Mã1']=df['Khối']
# chọn các cột cần thiết
filtered_df=df[['Mã1','Tháng 1','Tháng 2','Tháng 3','Tháng 4','Tháng 5','Tháng 6','Tháng 7','Tháng 8','Tháng 9',
               'Tháng 10','Tháng 11','Tháng 12','Tổng cộng']].rename(columns={'Mã1':'Mã'})
filtered_df=filtered_df[filtered_df['Mã'].notnull()]
melted_df=pd.melt(filtered_df, id_vars=['Mã'], var_name='Month', value_name='tar')
melted_df['Month']=melted_df['Month'].str.replace("Tháng ","")
# số liệu kế hoạch
df2=melted_df.copy()
final_KH_df=melted_df.copy()
# chỉnh lại các mã trong file kế hoạch cho giống file thực hiện
df2.loc[df2['Mã']=='CP lương kinh doanh bảo hiểm','Mã']='Chi phí lương kinh doanh bảo hiểm'
df2.loc[df2['Mã']=='Bảo hiểm tiền gửi','Mã']='Chi phí bảo hiểm, bảo toàn tiền gửi'

# lọc ra số tháng trong file kế hoạch
SoThang=len(df1['Month'].unique())-1
df2=df2[df2['Month']!='Tổng cộng']
df2['Month']=df2['Month'].astype(int)
CacThang_df2=df2[df2['Month']<=SoThang]
LuyKe_df2=CacThang_df2.groupby(['Mã']).tar.sum().reset_index()
LuyKe_df2['Month']='Lũy kế'

# kết hợp file kế hoạch các tháng và lũy kế
final_KH_df=pd.concat([CacThang_df2,LuyKe_df2])
final_KH_df['tar']=final_KH_df['tar'].round(2)

final_KH_df.Month=final_KH_df.Month.astype(str)
df2=final_KH_df.copy()

# GHÉP SỐ LIỆU KẾ HOẠCH VÀ SỐ THỰC HIỆN

In [49]:
merged_df=pd.merge(df1,df2,on=['Month','Mã'])
merged_df=merged_df.rename(columns={'val':'Thực hiện -','tar':'Kế hoạch -'})

In [50]:
# tạo cột % thực hiện so với KH
merged_df['% so KH - ']=(merged_df['Kế hoạch -']/merged_df['Thực hiện -']*100).round()
df=merged_df.copy()
# thay thế các giá trị inf vs 0
df.replace(np.inf, 0, inplace=True)
# chuyển cột phần trăm thành số nguyên và thêm kí hiệu %
df['% so KH -']=df['% so KH - '].astype(int).astype(str)+"%"
df['Month'] = df['Month'].apply(lambda x: 'Tháng ' + str(x) if x != 'Lũy kế' else x)


In [51]:
df=df.drop_duplicates()
# xoay dữ liệu thành các tháng hàng ngang
pivot_df=df.pivot(index='Mã', columns='Month', values=['Thực hiện -','Kế hoạch -','% so KH -']).reset_index()
# Flatten the multi-level column index and convert integers to strings
pivot_df.columns = [' '.join(map(str, col)).strip() for col in pivot_df.columns.values]

# sắp xếp các cột THỰC HIỆN, KẾ HOẠCH, % SO K
# tạo danh sách các list KH, TH và %
KH_columns = [col for col in pivot_df.columns if "Kế hoạch" in col]
index_to_move = KH_columns.index('Kế hoạch - Lũy kế')
KH_columns.pop(index_to_move)
KH_columns.append('Kế hoạch - Lũy kế')

TH_columns = [col for col in pivot_df.columns if "Thực hiện" in col]
index_to_move = TH_columns.index('Thực hiện - Lũy kế')
TH_columns.pop(index_to_move)
TH_columns.append('Thực hiện - Lũy kế')

PCT_columns = [col for col in pivot_df.columns if "% so KH" in col]
index_to_move = PCT_columns.index('% so KH - Lũy kế')
PCT_columns.pop(index_to_move)
PCT_columns.append('% so KH - Lũy kế')

# sắp xếp các cột theo thứ tự thực hiện, kế hoạch, % so KH
combined_columns = [a for pair in zip(TH_columns, KH_columns, PCT_columns) for a in pair]

df2=pivot_df[combined_columns].round(2).fillna(0)
df1=pivot_df['Mã']
df=pd.concat([df1, df2], axis=1)


# SẮP XẾP THEO THỨ TỰ

In [52]:
custom_order = ['HO_010','HO_020','HO_030','HO_040','HO_050','HO_060','HO_070','HO_500','HO_600','HO_613','HO_630','HO_700',
'HO_740','HO_750','HO_200','HO_100','HO_800','HO_300','HO_350','HO_400','HO_990','HO_640','HO_668','HO_CHUNG','HO_KV','DVKD',
'Tổng chi phí quản lý','Chi phí lương kinh doanh bảo hiểm','Chi phí bảo hiểm, bảo toàn tiền gửi','Tổng chi phí hoạt động']
# Create a custom sorting key based on the custom order
df['sorting_key'] = df['Mã'].apply(lambda x: custom_order.index(x))

# Sort the DataFrame by the custom sorting key
sorted_df = df.sort_values(by='sorting_key')



In [53]:
# HOÀN THÀNH 1 SHEET KHỐI THÁNG
Khoi_Thang_df=sorted_df.copy().iloc[:,0:-1] # loại bỏ cột sorting_key

In [54]:
dictionary = {
    'HO_010': 'Hội đồng quản trị & ban kiểm soát',
    'HO_020': 'Phòng kiểm toán nội bộ',
    'HO_030': 'Ban Tổng Giám đốc',
    'HO_040': 'VPLĐ, VP HĐQT, đầu tư, đối ngoại, chiến lược',
    'HO_050': 'Ban pháp chế & Kiểm soát tuân thủ',
    'HO_060': 'Khối tài chính kế hoạch',
    'HO_070': 'Khối nhân sự',
    'HO_500': 'Phòng Marketing',
    'HO_600': 'Khối vận hành',
    'HO_613': 'Trung tâm QL & HTTD',
    'HO_630': 'Khối CNTT & NHĐT',
    'HO_700': 'Khối quản trị rủi ro',
    'HO_740': 'Trung tâm XLN Miền Nam',
    'HO_750': 'Tung tâm XLN Miền Bắc',
    'HO_200': 'Khối Khách hàng cá nhân',
    'HO_100': 'Khối Khách hàng doanh nghiệp',
    'HO_800': 'Khối KHDNL & Định chế tài chính',
    'HO_300': 'Khối nguồn vốn & kinh doanh tiền tệ',
    'HO_350': 'Phòng đầu tư',
    'HO_400': 'Trung tâm thẻ',
    'HO_990': 'Khối Ngân hàng bảo hiểm',
    'HO_640': 'Trung tâm chuyển đổi số',
    'HO_668': 'Đội DA Mua trước trả sau (BNPL)',
    'HO_CHUNG': 'Chi phí chung tại HO',
    'HO_KV': 'Chi phí chung tại các Khu vực',
    'DVKD': 'Kênh phân phối',
    
}

Khoi_Thang_df['Khối'] = Khoi_Thang_df['Mã'].map(dictionary)
# Rearrange the columns to place 'description' at position 1
Khoi_Thang_df = Khoi_Thang_df[['Mã', 'Khối'] + [col for col in Khoi_Thang_df.columns if col != 'Mã' and col != 'Khối']]

# 4. KHỐI - QUÝ

In [55]:
# bảng số liệu thực hiện 
df=final_TH_df.rename(columns={'val':'Thực hiện -'})
# loại bỏ cột lũy kế theo tháng
df=df[df['Month']!="Lũy kế"]
# thêm cột quý
df['Month']=df['Month'].astype(int)


# Define a function to map months to quarters
def map_month_to_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    else:
        return 4
# Apply the mapping function to create the 'Quarter' column
df['Quarter'] = df['Month'].apply(map_month_to_quarter)

df=df.groupby(['Mã','Quarter'])['Thực hiện -'].sum().reset_index()
df['CumulativeSum']=df.groupby('Mã')['Thực hiện -'].transform('cumsum')

df1=df[['Mã','Quarter','Thực hiện -']]
df1['Quarter']="Quý "+df1['Quarter'].astype(str)
df2=df[['Mã','Quarter','CumulativeSum']].rename(columns={'CumulativeSum':'Thực hiện -'})
df2['Quarter']="Lũy kế quý "+df2['Quarter'].astype(str)

# Số liệu thực hiện theo quý
modified_TH_df1=pd.concat([df1,df2])
modified_TH_df1

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\961421892.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Quarter']="Quý "+df1['Quarter'].astype(str)


,Mã,Quarter,Thực hiện -
0,"Chi phí bảo hiểm, bảo toàn tiền gửi",Quý 1,50.67
1,"Chi phí bảo hiểm, bảo toàn tiền gửi",Quý 2,61.05
2,"Chi phí bảo hiểm, bảo toàn tiền gửi",Quý 3,26.44
3,Chi phí lương kinh doanh bảo hiểm,Quý 1,60.55
4,Chi phí lương kinh doanh bảo hiểm,Quý 2,109.95
...,...,...,...
85,Tổng chi phí hoạt động,Lũy kế quý 2,3100.27
86,Tổng chi phí hoạt động,Lũy kế quý 3,3686.89
87,Tổng chi phí quản lý,Lũy kế quý 1,1289.58
88,Tổng chi phí quản lý,Lũy kế quý 2,2818.04


In [56]:
# bảng số liệu thực hiện 
df=final_KH_df.rename(columns={'tar':'Kế hoạch -'})
# loại bỏ cột lũy kế theo tháng
df=df[df['Month']!="Lũy kế"]
# thêm cột quý
df['Month']=df['Month'].astype(int)

# Define a function to map months to quarters
def map_month_to_quarter(month):
    if month in [1, 2, 3]:
        return 1
    elif month in [4, 5, 6]:
        return 2
    elif month in [7, 8, 9]:
        return 3
    else:
        return 4
# Apply the mapping function to create the 'Quarter' column
df['Quarter'] = df['Month'].apply(map_month_to_quarter)

df=df.groupby(['Mã','Quarter'])['Kế hoạch -'].sum().reset_index()
df['CumulativeSum']=df.groupby('Mã')['Kế hoạch -'].transform('cumsum')
df1=df[['Mã','Quarter','Kế hoạch -']]
df1['Quarter']="Quý "+df1['Quarter'].astype(str)
df2=df[['Mã','Quarter','CumulativeSum']].rename(columns={'CumulativeSum':'Kế hoạch -'})
df2['Quarter']="Lũy kế quý "+df2['Quarter'].astype(str)

# Số liệu thực hiện theo quý
modified_KH_df1=pd.concat([df1,df2])
modified_KH_df1

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\3153412877.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Quarter']="Quý "+df1['Quarter'].astype(str)


,Mã,Quarter,Kế hoạch -
0,"Chi phí bảo hiểm, bảo toàn tiền gửi",Quý 1,57.99
1,"Chi phí bảo hiểm, bảo toàn tiền gửi",Quý 2,57.99
2,"Chi phí bảo hiểm, bảo toàn tiền gửi",Quý 3,19.33
3,Chi phí lương kinh doanh bảo hiểm,Quý 1,160.11
4,Chi phí lương kinh doanh bảo hiểm,Quý 2,160.11
...,...,...,...
88,Tổng chi phí hoạt động,Lũy kế quý 2,3687.38
89,Tổng chi phí hoạt động,Lũy kế quý 3,4266.48
90,Tổng chi phí quản lý,Lũy kế quý 1,1553.71
91,Tổng chi phí quản lý,Lũy kế quý 2,3201.16


# GHÉP SỐ KẾ HOẠCH VÀ SỐ THỰC HIỆN

In [57]:
merged_df=pd.merge(modified_TH_df1,modified_KH_df1,on=['Mã','Quarter'])
# tạo cột % thực hiện so với KH
merged_df['% so KH -']=(merged_df['Kế hoạch -']/merged_df['Thực hiện -']*100).round()
df=merged_df.copy()
# thay thế các giá trị inf vs 0
df.replace(np.inf, 0, inplace=True)
# chuyển cột phần trăm thành số nguyên và thêm kí hiệu %
df['% so KH -']=df['% so KH -'].astype(int).astype(str)+"%"
df=df.sort_values(by='Quarter')

# Xoay dữ liệu thành chiều ngang với 
# Example pivot operation, replace this with your actual pivot operation
pivot_df = df.pivot(index='Mã', columns='Quarter', values=['Kế hoạch -', 'Thực hiện -', '% so KH -']).reset_index()
# Flatten the multi-level column index and convert integers to strings
pivot_df.columns = [' '.join(map(str, col)).strip() for col in pivot_df.columns.values]
pivot_df.columns

KH_columns1 = [col for col in pivot_df.columns if "Kế hoạch - Lũy" in col]
KH_columns2 = [col for col in pivot_df.columns if "Kế hoạch - Quý" in col]
KH_columns=KH_columns2+KH_columns1


TH_columns1 = [col for col in pivot_df.columns if "Thực hiện - Lũy" in col]
TH_columns2 = [col for col in pivot_df.columns if "Thực hiện - Quý" in col]
TH_columns=TH_columns2+TH_columns1


PCT_columns1 = [col for col in pivot_df.columns if "% so KH" in col]
PCT_columns2 = [col for col in pivot_df.columns if "% so KH" in col]
PCT_columns=PCT_columns2+PCT_columns1

# sắp xếp các cột theo thứ tự thực hiện, kế hoạch, % so KH
combined_columns = [a for pair in zip(TH_columns, KH_columns, PCT_columns) for a in pair]
df2=pivot_df[combined_columns].round(2).fillna(0)
df1=pivot_df['Mã']
df=pd.concat([df1, df2], axis=1)


# SẮP XẾP THEO THỨ TỰ

In [58]:
custom_order = ['HO_010','HO_020','HO_030','HO_040','HO_050','HO_060','HO_070','HO_500','HO_600','HO_613','HO_630','HO_700',
'HO_740','HO_750','HO_200','HO_100','HO_800','HO_300','HO_350','HO_400','HO_990','HO_640','HO_668','HO_CHUNG','HO_KV','DVKD',
'Tổng chi phí quản lý','Chi phí lương kinh doanh bảo hiểm','Chi phí bảo hiểm, bảo toàn tiền gửi','Tổng chi phí hoạt động']
# Create a custom sorting key based on the custom order
df['sorting_key'] = df['Mã'].apply(lambda x: custom_order.index(x))

# Sort the DataFrame by the custom sorting key
sorted_df = df.sort_values(by='sorting_key')

In [59]:
# HOÀN THÀNH 1 SHEET KHỐI - QUÝ
Khoi_Quy_df=sorted_df.iloc[:,0:-1].copy()

In [60]:
dictionary = {
    'HO_010': 'Hội đồng quản trị & ban kiểm soát',
    'HO_020': 'Phòng kiểm toán nội bộ',
    'HO_030': 'Ban Tổng Giám đốc',
    'HO_040': 'VPLĐ, VP HĐQT, đầu tư, đối ngoại, chiến lược',
    'HO_050': 'Ban pháp chế & Kiểm soát tuân thủ',
    'HO_060': 'Khối tài chính kế hoạch',
    'HO_070': 'Khối nhân sự',
    'HO_500': 'Phòng Marketing',
    'HO_600': 'Khối vận hành',
    'HO_613': 'Trung tâm QL & HTTD',
    'HO_630': 'Khối CNTT & NHĐT',
    'HO_700': 'Khối quản trị rủi ro',
    'HO_740': 'Trung tâm XLN Miền Nam',
    'HO_750': 'Tung tâm XLN Miền Bắc',
    'HO_200': 'Khối Khách hàng cá nhân',
    'HO_100': 'Khối Khách hàng doanh nghiệp',
    'HO_800': 'Khối KHDNL & Định chế tài chính',
    'HO_300': 'Khối nguồn vốn & kinh doanh tiền tệ',
    'HO_350': 'Phòng đầu tư',
    'HO_400': 'Trung tâm thẻ',
    'HO_990': 'Khối Ngân hàng bảo hiểm',
    'HO_640': 'Trung tâm chuyển đổi số',
    'HO_668': 'Đội DA Mua trước trả sau (BNPL)',
    'HO_CHUNG': 'Chi phí chung tại HO',
    'HO_KV': 'Chi phí chung tại các Khu vực',
    'DVKD': 'Kênh phân phối',
    
}

Khoi_Quy_df['Khối'] = Khoi_Quy_df['Mã'].map(dictionary)
# Rearrange the columns to place 'description' at position 1
Khoi_Quy_df = Khoi_Quy_df[['Mã', 'Khối'] + [col for col in Khoi_Quy_df.columns if col != 'Mã' and col != 'Khối']]

# LƯU RA FILE EXCEL CHỨA TẤT CẢ CÁC DATA

In [61]:
import pandas as pd

# Specify the file path for the Excel file
excel_file_path = f'Phan-Tich-Chi-Phi-Thang {file_num}.xlsx'
writer = pd.ExcelWriter(excel_file_path, engine='xlsxwriter')

# Create a Pandas ExcelWriter object
with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    ChiPhi_HO_df.to_excel(writer, sheet_name='HO', startrow=0, startcol=0, index=False)
    ChiPhi_DVKD_df.to_excel(writer, sheet_name='DVKD', startrow=0, startcol=0, index=False)
    KhoanMuc_Thang_df.to_excel(writer, sheet_name='Khoan muc-Thang', startrow=0, startcol=0, index=False)
    Khoi_Thang_df.to_excel(writer, sheet_name='Khoi-Thang', startrow=0, startcol=0, index=False)
    KhoanMuc_Quy_df.to_excel(writer, sheet_name='Khoan muc-Quy', startrow=0, startcol=0, index=False)
    Khoi_Quy_df.to_excel(writer, sheet_name='Khoi-Quy', startrow=0, startcol=0, index=False)
    
    # ĐỒ THỊ THEO THÁNG
    # Write the first DataFrame (do thi khoan muc thang) starting from cell A1
    KhoanMuc_Thang_chart_df.to_excel(writer, sheet_name='Do thi_theo thang', startrow=0, startcol=0, index=False)
    # Calculate the number of rows in df1 and add 20 to determine the start row for df2
    start_row_df2 = KhoanMuc_Thang_chart_df.shape[0] + 10
    # Write the second DataFrame (do thi khoan muc thang theo pct) starting from cell A20
    KhoanMuc_Thang_chart_in_pct_df.to_excel(writer, sheet_name='Do thi_theo thang', startrow=start_row_df2, startcol=0, index=False)
    # Write the second DataFrame (do thi khoan muc thang theo pct) starting from cell A30
    start_row_df3 = KhoanMuc_Thang_chart_in_pct_df.shape[0] + 10
    KhoanMuc_Thang_So_KH_df.to_excel(writer, sheet_name='Do thi_theo thang', startrow=start_row_df3, startcol=0, index=False)
    
    
    # ĐỒ THỊ THEO THÁNG ENG VER
    # Write the first DataFrame (do thi khoan muc thang) starting from cell A1
    KhoanMuc_Thang_chart_df_Eng.to_excel(writer, sheet_name='Do thi_theo thang-EN', startrow=0, startcol=0, index=False)
    # Calculate the number of rows in df1 and add 20 to determine the start row for df2
    start_row_df2 = KhoanMuc_Thang_chart_df_Eng.shape[0] + 10
    # Write the second DataFrame (do thi khoan muc thang theo pct) starting from cell A20
    KhoanMuc_Thang_chart_in_pct_df_Eng.to_excel(writer, sheet_name='Do thi_theo thang-EN', startrow=start_row_df2, startcol=0, index=False)
    # Write the second DataFrame (do thi khoan muc thang theo pct) starting from cell A30
    start_row_df3 = KhoanMuc_Thang_chart_in_pct_df.shape[0] + 10
    KhoanMuc_Thang_So_KH_df_Eng.to_excel(writer, sheet_name='Do thi_theo thang-EN', startrow=start_row_df3, startcol=0, index=False)
    
    # ĐỒ THỊ THEO QUÝ
    # Write the first DataFrame (do thi khoan muc quy) starting from cell A1
    KhoanMuc_Quy_chart_df.to_excel(writer, sheet_name='Do thi_theo quy', startrow=0, startcol=0, index=False)
    # Calculate the number of rows in df1 and add 20 to determine the start row for df2
    start_row_df2 = KhoanMuc_Quy_chart_df.shape[0] + 10
    # Write the second DataFrame (do thi khoan muc quý theo pct) starting from cell A20
    KhoanMuc_Quy_chart_in_pct_df.to_excel(writer, sheet_name='Do thi_theo quy', startrow=start_row_df2, startcol=0, index=False)
    
    # ĐỒ THỊ THEO QUÝ ENG VER
    # Write the first DataFrame (do thi khoan muc quy) starting from cell A1
    KhoanMuc_Quy_chart_df_Eng.to_excel(writer, sheet_name='Do thi_theo quy-EN', startrow=0, startcol=0, index=False)
    # Calculate the number of rows in df1 and add 20 to determine the start row for df2
    start_row_df2 = KhoanMuc_Quy_chart_df_Eng.shape[0] + 10
    # Write the second DataFrame (do thi khoan muc quy theo pct) starting from cell A20
    KhoanMuc_Quy_chart_in_pct_df_Eng.to_excel(writer, sheet_name='Do thi_theo quy-EN', startrow=start_row_df2, startcol=0, index=False)
        
# Save the Excel file
writer.save()

# Release the Excel file
writer.close()

C:\Users\phatdd3\AppData\Local\Temp\ipykernel_2940\3666753577.py:49: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
C:\Users\phatdd3\AppData\Local\anaconda3\lib\site-packages\xlsxwriter\workbook.py:339: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [62]:
print('your data is ready now')

your data is ready now


# FORMAT LẠI FILE EXCEL CHO BÁO CÁO

In [63]:
from openpyxl import load_workbook
wb=load_workbook(excel_file_path)


In [85]:
from openpyxl.styles import numbers
column_widths = {'A': 14, 'B': 8, 'C': 16, 'D': 15, 'E': 35}

# Define the list of sheet names
sheets = ['HO', 'DVKD']


# Process each sheet
for sheet_name in sheets:
    ws = wb[sheet_name]
    for column, width in column_widths.items():
        ws.column_dimensions[column].width = width
        
    for cell in ws[1]:
        cell.alignment = openpyxl.styles.Alignment(wrapText=True)
    # Find the maximum column index
    max_column_index = ws.max_column
    
    # Set the column width for columns F to the max column to 15
    for column_index in range(6, max_column_index + 1):  # 6 corresponds to column F
        column_letter = openpyxl.utils.get_column_letter(column_index)
        ws.column_dimensions[column_letter].width = 15
        
    max_row = ws.max_row
    
    # Set the decimal thousand separator for the range F2 to the max column and max row
    for row in ws.iter_rows(min_row=2, max_row=max_row, min_col=6, max_col=max_column_index):
        for cell in row:
            cell.number_format = '#,##0'
        
wb.save(excel_file_path)


## KHOẢN MỤC - THÁNG

In [65]:
from openpyxl import load_workbook
# mở file excel 
wb=load_workbook(excel_file_path)
# Select the worksheet
ws = wb['Khoan muc-Thang']

In [66]:
from openpyxl.utils import get_column_letter
# Set the column width
column_widths = {'A': 5, 'B': 40}

for column, width in column_widths.items():
    ws.column_dimensions[column].width = width

# Find the maximum column index
max_column_index = ws.max_column

# Set max column width for columns C to the maximum column
max_column_width = 12

for column_index in range(3, max_column_index + 1):  # Starting from column C
    column_letter = get_column_letter(column_index)
    ws.column_dimensions[column_letter].width = max_column_width
    
# Apply wrap text format to the cells in the first row
for cell in ws[1]:
    cell.alignment = openpyxl.styles.Alignment(wrapText=True)
    
wb.save(excel_file_path)


In [67]:
from openpyxl.styles import Font, colors
from openpyxl.styles import PatternFill


# List of rows you want to make bold
rows_to_bold = [2, 3, 4, 20, 26, 27, 30]

# Define the font style (bold)
bold_font = Font(bold=True)

# Define the fill color
fill_color = PatternFill(start_color="FDE9D9", end_color="FDE9D9", fill_type="solid")

# Loop through the rows and set the font
for row_number in rows_to_bold:
    
    for cell in ws[row_number]:
        cell.font = bold_font
        cell.fill = fill_color
        
    for cell in ws[30]:
        cell.fill = PatternFill(start_color="ffff00", end_color="ffff00", fill_type="solid")

# Save the modified workbook
wb.save(excel_file_path)

In [68]:
def fill_first_row_with_color(worksheet, start_column, end_column, color_code):
    # Define the fill color
    fill_color = PatternFill(start_color=color_code, end_color=color_code, fill_type="solid")
    
    # Loop through the specified range of columns in the first row
    for column in range(start_column, end_column + 1):
        cell = worksheet.cell(row=1, column=column)
        cell.fill = fill_color
    # Save the workbook
    wb.save(excel_file_path)
    
count = file_num+1  # Change this value to the desired count
increment = 3
start_value = 3  # Change this value if you want a different starting point
start_col_list = [start_value + i * increment for i in range(count)]

increment = 3
start_value = 6  # Change this value if you want a different starting point
end_col_list = [start_value + i * increment for i in range(count)]

In [69]:

# List of aRGB hex color codes for each category
category_color_codes = [
    "F2C9CC", "F2D5B0", "F2E0B3", "E8E597", "E9E4B4", "CBE6E9", "CED4B3", "E1D5E7", "D9E5D6", "F7D7E5", 
    "F1E0CC", "F7E1D7", "E0E3E4" ]

# Create the dictionary
data_dict = {
    'start_col_list': start_col_list,
    'end_col_list': end_col_list,
    'category_color_codes': category_color_codes
}

for start_col, end_col, color_code in zip(data_dict['start_col_list'], data_dict['end_col_list'], data_dict['category_color_codes']):
    fill_first_row_with_color(ws, start_col, end_col, color_code)

In [70]:
from openpyxl.utils import get_column_letter
# create some grouping buttons
pairs = [
    ('D', 'E'),
    ('G', 'H'),
    ('J', 'K'),
    ('M', 'N'),
    ('P', 'Q'),
    ('S', 'T'),
    ('V', 'W'),
    ('Y', 'Z')
]

for start_col, end_col in pairs:
    ws.column_dimensions.group(start=start_col, end=end_col, hidden=True)

wb.save(excel_file_path)


## KHỐI - THÁNG

In [71]:
# Select the worksheet
ws = wb['Khoi-Thang']

# Merge cells A28 to A31
ws.merge_cells('A28:B28')
ws.merge_cells('A29:B29')
ws.merge_cells('A30:B30')
ws.merge_cells('A31:B31')

# Set the column width
column_widths = {'A': 12, 'B': 40}

for column, width in column_widths.items():
    ws.column_dimensions[column].width = width

# Find the maximum column index
max_column_index = ws.max_column

# Set max column width for columns C to the maximum column
max_column_width = 12

for column_index in range(3, max_column_index + 1):  # Starting from column C
    column_letter = get_column_letter(column_index)
    ws.column_dimensions[column_letter].width = max_column_width
wb.save(excel_file_path)
    
# Apply wrap text format to the cells in the first row
for cell in ws[1]:
    cell.alignment = openpyxl.styles.Alignment(wrapText=True)
    
# List of rows you want to make bold
rows_to_bold = [28, 31]

# Define the font style (bold)
bold_font = Font(bold=True)

# Loop through the rows and set the font
for row_number in rows_to_bold:
    
    for cell in ws[row_number]:
        cell.font = bold_font
        
wb.save(excel_file_path)

In [72]:
# create some grouping buttons
pairs = [
    ('D', 'E'),
    ('G', 'H'),
    ('J', 'K'),
    ('M', 'N'),
    ('P', 'Q'),
    ('S', 'T'),
    ('V', 'W'),
    ('Y', 'Z')
]

for start_col, end_col in pairs:
    ws.column_dimensions.group(start=start_col, end=end_col, hidden=True)

wb.save(excel_file_path)

In [73]:
count = file_num+1  # Change this value to the desired count
increment = 3
start_value = 3  # Change this value if you want a different starting point
start_col_list = [start_value + i * increment for i in range(count)]

increment = 3
start_value = 6  # Change this value if you want a different starting point
end_col_list = [start_value + i * increment for i in range(count)]

# List of aRGB hex color codes for each category
category_color_codes = [
    "F2C9CC", "F2D5B0", "F2E0B3", "E8E597", "E9E4B4", "CBE6E9", "CED4B3", "E1D5E7", "D9E5D6", "F7D7E5", 
    "F1E0CC", "F7E1D7", "E0E3E4" ]

# Create the dictionary
data_dict = {
    'start_col_list': start_col_list,
    'end_col_list': end_col_list,
    'category_color_codes': category_color_codes
}

for start_col, end_col, color_code in zip(data_dict['start_col_list'], data_dict['end_col_list'], data_dict['category_color_codes']):
    fill_first_row_with_color(ws, start_col, end_col, color_code)

wb.save(excel_file_path)

# Khoan muc-Quy

In [74]:
# Select the worksheet
ws = wb['Khoan muc-Quy']

In [75]:
from openpyxl.utils import get_column_letter
# Set the column width
column_widths = {'A': 5, 'B': 40}

for column, width in column_widths.items():
    ws.column_dimensions[column].width = width

# Find the maximum column index
max_column_index = ws.max_column

# Set max column width for columns C to the maximum column
max_column_width = 12

for column_index in range(3, max_column_index + 1):  # Starting from column C
    column_letter = get_column_letter(column_index)
    ws.column_dimensions[column_letter].width = max_column_width
    
# Apply wrap text format to the cells in the first row
for cell in ws[1]:
    cell.alignment = openpyxl.styles.Alignment(wrapText=True)

from openpyxl.styles import Font, colors
from openpyxl.styles import PatternFill


# List of rows you want to make bold
rows_to_bold = [2, 3, 4, 20, 26, 27, 30]

# Define the font style (bold)
bold_font = Font(bold=True)

# Define the fill color
fill_color = PatternFill(start_color="FDE9D9", end_color="FDE9D9", fill_type="solid")

# Loop through the rows and set the font
for row_number in rows_to_bold:
    
    for cell in ws[row_number]:
        cell.font = bold_font
        cell.fill = fill_color
        
    for cell in ws[30]:
        cell.fill = PatternFill(start_color="ffff00", end_color="ffff00", fill_type="solid")

# Save the modified workbook
wb.save(excel_file_path)



In [76]:
count = file_num+1  # Change this value to the desired count
increment = 3
start_value = 3  # Change this value if you want a different starting point
start_col_list = [start_value + i * increment for i in range(count)]

increment = 3
start_value = 6  # Change this value if you want a different starting point
end_col_list = [start_value + i * increment for i in range(count)]


# List of aRGB hex color codes for each category
category_color_codes = [
    "F2C9CC", "F2D5B0", "F2E0B3", "E8E597", "E9E4B4", "CBE6E9", "CED4B3", "E1D5E7", "D9E5D6", "F7D7E5", 
    "F1E0CC", "F7E1D7", "E0E3E4" ]

# Create the dictionary
data_dict = {
    'start_col_list': start_col_list,
    'end_col_list': end_col_list,
    'category_color_codes': category_color_codes
}

for start_col, end_col, color_code in zip(data_dict['start_col_list'], data_dict['end_col_list'], data_dict['category_color_codes']):
    fill_first_row_with_color(ws, start_col, end_col, color_code)
    
from openpyxl.utils import get_column_letter
# create some grouping buttons
pairs = [
    ('D', 'E'),
    ('G', 'H'),
    ('J', 'K'),
    ('M', 'N'),
    ('P', 'Q'),
    ('S', 'T'),
    ('V', 'W'),
]

for start_col, end_col in pairs:
    ws.column_dimensions.group(start=start_col, end=end_col, hidden=True)

wb.save(excel_file_path)


# Khoi-Quy

In [77]:
# Select the worksheet
ws = wb['Khoi-Quy']

In [78]:
# Merge cells A28 to A31
ws.merge_cells('A28:B28')
ws.merge_cells('A29:B29')
ws.merge_cells('A30:B30')
ws.merge_cells('A31:B31')

# Set the column width
column_widths = {'A': 12, 'B': 40}

for column, width in column_widths.items():
    ws.column_dimensions[column].width = width

# Find the maximum column index
max_column_index = ws.max_column

# Set max column width for columns C to the maximum column
max_column_width = 12

for column_index in range(3, max_column_index + 1):  # Starting from column C
    column_letter = get_column_letter(column_index)
    ws.column_dimensions[column_letter].width = max_column_width
wb.save(excel_file_path)
    
# Apply wrap text format to the cells in the first row
for cell in ws[1]:
    cell.alignment = openpyxl.styles.Alignment(wrapText=True)
    
# List of rows you want to make bold
rows_to_bold = [28, 31]

# Define the font style (bold)
bold_font = Font(bold=True)

# Loop through the rows and set the font
for row_number in rows_to_bold:
    
    for cell in ws[row_number]:
        cell.font = bold_font
        
wb.save(excel_file_path)

In [79]:
# create some grouping buttons
pairs = [
    ('D', 'E'),
    ('G', 'H'),
    ('J', 'K'),
    ('M', 'N'),
    ('P', 'Q'),
    ('S', 'T'),
    ('V', 'W'),
    ('Y', 'Z')
]

for start_col, end_col in pairs:
    ws.column_dimensions.group(start=start_col, end=end_col, hidden=True)

wb.save(excel_file_path)

In [80]:
count = file_num+1  # Change this value to the desired count
increment = 3
start_value = 3  # Change this value if you want a different starting point
start_col_list = [start_value + i * increment for i in range(count)]

increment = 3
start_value = 6  # Change this value if you want a different starting point
end_col_list = [start_value + i * increment for i in range(count)]

# List of aRGB hex color codes for each category
category_color_codes = [
    "F2C9CC", "F2D5B0", "F2E0B3", "E8E597", "E9E4B4", "CBE6E9", "CED4B3", "E1D5E7", "D9E5D6", "F7D7E5", 
    "F1E0CC", "F7E1D7", "E0E3E4" ]

# Create the dictionary
data_dict = {
    'start_col_list': start_col_list,
    'end_col_list': end_col_list,
    'category_color_codes': category_color_codes
}

for start_col, end_col, color_code in zip(data_dict['start_col_list'], data_dict['end_col_list'], data_dict['category_color_codes']):
    fill_first_row_with_color(ws, start_col, end_col, color_code)

wb.save(excel_file_path)

# CÁC SHEET ĐỒ THỊ

In [81]:
import openpyxl
from openpyxl.styles import Alignment, numbers
# Define a function to set column width and enable text wrapping for a given sheet
def set_column_width_and_wrap_text(sheet, column_widths):
    for column, width in column_widths.items():
        sheet.column_dimensions[column].width = width
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=1):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

# Define a function to format a range as percentages for a given sheet
def format_as_percent(sheet, start_row, start_column, end_row, end_column):
    for row in sheet.iter_rows(min_row=start_row, max_row=end_row, min_col=start_column, max_col=end_column):
        for cell in row:
            cell.number_format = numbers.FORMAT_PERCENTAGE

def fill_row_with_color(sheet, row, end_col, color="FF948A54"):
    # Define the color you want to fill
    fill_color = PatternFill(start_color=color, end_color=color, fill_type='solid')

    # Loop through the columns and apply the fill color
    for col in range(1, end_col + 1):
        cell = sheet.cell(row=row, column=col)
        cell.fill = fill_color

In [82]:
# Define the list of sheet names
sheets = ['Do thi_theo thang', 'Do thi_theo thang-EN']

# Set the column width and enable text wrapping for column A
column_widths = {'A': 40}

# Define the range (B19:I25) you want to format as percent
start_row, start_column, end_row, end_column = 19, 2, 25, file_num + 2

# Process each sheet
for sheet_name in sheets:
    sheet = wb[sheet_name]
    
    # Set column width and enable text wrapping for column A
    set_column_width_and_wrap_text(sheet, column_widths)
    fill_color = PatternFill(start_color='948a54', end_color='948a54', fill_type='solid')

    # Format the specified range as percentages
    format_as_percent(sheet, start_row, start_column, end_row, end_column)
    
    # Format the specified range to fill color 948a54
    end_col = file_num+2
    
    fill_row_with_color(sheet, 1, end_col, color="FF948A54")
    fill_row_with_color(sheet, 18, end_col, color="FF948A54")

# Save the modified Excel file
wb.save(excel_file_path)

# Close the workbook
wb.close()


In [83]:
import openpyxl
from openpyxl.styles import Alignment, numbers

# Define a function to set column width and enable text wrapping for a given sheet
def set_column_width_and_wrap_text(sheet, column_widths):
    for column, width in column_widths.items():
        sheet.column_dimensions[column].width = width
    for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row, min_col=1, max_col=1):
        for cell in row:
            cell.alignment = Alignment(wrap_text=True)

# Define a function to format a range as percentages for a given sheet
def format_as_percent(sheet, start_row, start_column, end_row, end_column):
    for row in sheet.iter_rows(min_row=start_row, max_row=end_row, min_col=start_column, max_col=end_column):
        for cell in row:
            cell.number_format = numbers.FORMAT_PERCENTAGE

# Define the list of sheet names
sheets = ['Do thi_theo quy', 'Do thi_theo quy-EN']

# Set the column width and enable text wrapping for column A
column_widths = {'A': 40}

# Define the range (B19:I25) you want to format as percent
start_row, start_column, end_row, end_column = 19, 2, 25, file_num//3+1+1

# Process each sheet
for sheet_name in sheets:
    sheet = wb[sheet_name]
    
    # Set column width and enable text wrapping for column A
    set_column_width_and_wrap_text(sheet, column_widths)
    
    # Format the specified range as percentages
    format_as_percent(sheet, start_row, start_column, end_row, end_column)
    
    end_col = file_num//3+1+1
    
    fill_row_with_color(sheet, 1, end_col, color="FF948A54")
    fill_row_with_color(sheet, 18, end_col, color="FF948A54")
    

# Save the modified Excel file
wb.save(excel_file_path)

# Close the workbook
wb.close()
